## 1. Methylation

In [ ]:
import pandas as pd

In [ ]:
raw_methy_df = pd.read_csv('./raw_data/CCLE_RRBS_TSS1kb_20181022.txt', sep='\t')
display(raw_methy_df)

In [ ]:
# column locus_id should be separate the content after the  '_'
raw_methy_df['locus_id'] = raw_methy_df['locus_id'].apply(lambda x: x.split('_')[0])
# drop na in the column locus_id
raw_methy_df = raw_methy_df.dropna(subset=['CpG_sites_hg19'])
# remove the column CpG_sites_hg19 and avg_coverage
raw_methy_df = raw_methy_df.drop(columns=['CpG_sites_hg19', 'avg_coverage'])
display(raw_methy_df)

In [ ]:
# Identify columns to convert (all except 'locus_id')
cols_to_convert = raw_methy_df.columns.difference(['locus_id'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_methy_df[cols_to_convert] = raw_methy_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_methy_df = raw_methy_df.fillna(0.0)
display(raw_methy_df)

In [ ]:
# Now perform the groupby mean operation
methy_df = raw_methy_df.groupby('locus_id', as_index=False).mean()
display(methy_df)

In [ ]:
bmg_promoter_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_promoter_df = bmg_promoter_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_promoter_df)

In [ ]:
# merge the biomedgraphica_id with the raw_methy_df
merged_methy_df = pd.merge(bmg_promoter_df, methy_df, left_on='HGNC_Symbol', right_on='locus_id', how='inner')
merged_methy_df.drop(columns=['HGNC_Symbol', 'locus_id'], inplace=True)
display(merged_methy_df)

In [ ]:
final_merged_methy_df = pd.merge(bmg_promoter_df, methy_df, left_on='HGNC_Symbol', right_on='locus_id', how='left')
# fill the NaN with 0.0
final_merged_methy_df = final_merged_methy_df.fillna(0.0)
final_merged_methy_df.drop(columns=['HGNC_Symbol', 'locus_id'], inplace=True)
display(final_merged_methy_df)

## 2. Gene

In [ ]:
raw_gene_df = pd.read_csv('./raw_data/OmicsCNGene.csv')
display(raw_gene_df)

In [ ]:
# First, set the "Unnamed: 0" column as the index:
raw_gene_t_df = raw_gene_df.set_index("Unnamed: 0")
# Then transpose the DataFrame:
raw_gene_t_df = raw_gene_t_df.transpose()
# move the index to a column and rename it as the gene_name
raw_gene_t_df.reset_index(inplace=True)
raw_gene_t_df.rename(columns={'index': 'gene_name'}, inplace=True)
# column gene_name should be kept with the content by removing "()" and remove and " " content
raw_gene_t_df['gene_name'] = raw_gene_t_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Optionally, if you want to view the result:
display(raw_gene_t_df)

In [ ]:
# Identify columns to convert (all except 'Unnamed: 0' and 'gene_name')
cols_to_convert = raw_gene_t_df.columns.difference(['Unnamed: 0', 'gene_name'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_gene_t_df[cols_to_convert] = raw_gene_t_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_gene_t_df = raw_gene_t_df.fillna(0.0)
display(raw_gene_t_df)

In [ ]:
# groupby the gene_name and calculate the mean
gene_df = raw_gene_t_df.groupby('gene_name', as_index=False).mean()
display(gene_df)

In [ ]:
bmg_gene_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_gene_df = bmg_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_gene_df)

In [ ]:
# merge the biomedgraphica_id with the raw_gene_t_df
merged_gene_df = pd.merge(bmg_gene_df, gene_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merged_gene_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merged_gene_df)

In [ ]:
final_merged_gene_df = pd.merge(bmg_gene_df, gene_df, left_on='HGNC_Symbol', right_on='gene_name', how='left')
# fill the NaN with -1.0
final_merged_gene_df = final_merged_gene_df.fillna(0.0)
final_merged_gene_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(final_merged_gene_df)

## 3. Transcript

In [ ]:
raw_transcript_df = pd.read_csv('./raw_data/OmicsExpressionProteinCodingGenesTPMLogp1BatchCorrected.csv')
display(raw_transcript_df)

In [ ]:
# First, set the "Unnamed: 0" column as the index:
raw_transcript_df = raw_transcript_df.set_index("Unnamed: 0")
# Then transpose the DataFrame:
raw_transcript_df = raw_transcript_df.transpose()
# move the index to a column and rename it as the gene_name
raw_transcript_df.reset_index(inplace=True)
raw_transcript_df.rename(columns={'index': 'gene_name'}, inplace=True)
# column gene_name should be kept with the content by removing "()" and remove and
raw_transcript_df['gene_name'] = raw_transcript_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Optionally, if you want to view the result:
display(raw_transcript_df)

In [ ]:
# Identify columns to convert (all except 'Unnamed: 0' and 'gene_name')
cols_to_convert = raw_transcript_df.columns.difference(['Unnamed: 0', 'gene_name'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_transcript_df[cols_to_convert] = raw_transcript_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_transcript_df = raw_transcript_df.fillna(0.0)
display(raw_transcript_df)

In [ ]:
bmg_transcript_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_transcript_df = bmg_transcript_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_transcript_df)

In [ ]:
# merge the biomedgraphica_id with the raw_transcript_df
merge_transcript_df = pd.merge(bmg_transcript_df, raw_transcript_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merge_transcript_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merge_transcript_df)

In [ ]:
# merge the biomedgraphica_id with the raw_transcript_df
final_merged_transcript_df = pd.merge(bmg_transcript_df, raw_transcript_df, left_on='HGNC_Symbol', right_on='gene_name', how='left')
# fill the NaN with -1.0
final_merged_transcript_df = final_merged_transcript_df.fillna(0.0)
final_merged_transcript_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(final_merged_transcript_df)

## 4. Protein

In [ ]:
raw_protein_df = pd.read_csv('./raw_data/protein_quant_current_normalized.csv')
raw_protein_df = raw_protein_df.drop(columns=['Protein_Id', 'Gene_Symbol', 'Description', 'Group_ID', 'Uniprot'])
# Also drop columns names contain Peptides
raw_protein_df = raw_protein_df[raw_protein_df.columns.drop(list(raw_protein_df.filter(regex='Peptides')))]
display(raw_protein_df)

In [ ]:
# Identify columns to convert (all except 'Unnamed: 0' and 'protein_name')
cols_to_convert = raw_protein_df.columns.difference(['Uniprot_Acc'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_protein_df[cols_to_convert] = raw_protein_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_protein_df = raw_protein_df.fillna(0.0)
display(raw_protein_df)

In [ ]:
# groupby the protein_name and calculate the mean
protein_df = raw_protein_df.groupby('Uniprot_Acc', as_index=False).mean()
display(protein_df)


In [ ]:
# merge the biomedgraphica_id with the raw_protein_df
bmg_protein_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')
bmg_protein_df = bmg_protein_df[['BioMedGraphica_Conn_ID', 'Uniprot_ID']]
display(bmg_protein_df)

In [ ]:
# merge the biomedgraphica_id with the raw_protein_df
merged_protein_df = pd.merge(bmg_protein_df, protein_df, left_on='Uniprot_ID', right_on='Uniprot_Acc', how='inner')
merged_protein_df.drop(columns=['Uniprot_ID', 'Uniprot_Acc'], inplace=True)
display(merged_protein_df)

In [ ]:
# merge the biomedgraphica_id with the raw_protein_df
final_merged_protein_df = pd.merge(bmg_protein_df, protein_df, left_on='Uniprot_ID', right_on='Uniprot_Acc', how='left')
# fill the NaN with 0.0
final_merged_protein_df = final_merged_protein_df.fillna(0.0)
final_merged_protein_df.drop(columns=['Uniprot_ID', 'Uniprot_Acc'], inplace=True)
display(final_merged_protein_df)

## 5. Drug

In [ ]:
raw_drug_df = pd.read_csv('./raw_data/sanger-dose-response.csv')
display(raw_drug_df)

In [ ]:
# keep columns ['ARXSPAN_ID', 'DRUG_NAME', 'IC50_PUBLISHED']
raw_drug_df = raw_drug_df[['ARXSPAN_ID', 'DRUG_NAME', 'IC50_PUBLISHED', 'AUC_PUBLISHED']]
# check if there is nan in all of the dataframe
print(raw_drug_df.isnull().sum())
# drop the nan in this dataframe
raw_drug_df = raw_drug_df.dropna().reset_index(drop=True)
# check if there is nan in all of the dataframe again
print(raw_drug_df.isnull().sum())
display(raw_drug_df)

In [ ]:
# fetch the drug_name as a independent dataframe
drug_name_df = raw_drug_df[['DRUG_NAME']]
# remove the duplicate in the drug_name_df
drug_name_df = drug_name_df.drop_duplicates().reset_index(drop=True)
display(drug_name_df)

In [ ]:
bmg_drug_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Drug/BioMedGraphica_Conn_Drug.csv')
display(bmg_drug_df)
bmg_drug_name_df = bmg_drug_df[['BioMedGraphica_Conn_ID', 'PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name', 'PubChem_Synonym']]
# keep BioMedGraphica_Conn_ID and Drug_Name
display(bmg_drug_name_df)

In [ ]:
# Combine 'PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name', 'PubChem_Synonym' into one column called 'Drug_Name'
bmg_drug_name_df['Drug_Name'] = bmg_drug_name_df[['PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name', 'PubChem_Synonym']].apply(lambda x: ';'.join(x.dropna().astype(str)), axis=1)
bmg_drug_name_df = bmg_drug_name_df[['BioMedGraphica_Conn_ID', 'Drug_Name']]
display(bmg_drug_name_df)

In [ ]:
# since in the column drug name, there are lots of names split by ";", if this space contain the name in the drug_name_df, then we should match this 2 rows and merge the drug_name_df with the bmg_drug_df
# Fix the drug name processing
# First, check if there are any NaN values in Drug_Name column
print(f"Number of NaN values in Drug_Name column: {bmg_drug_name_df['Drug_Name'].isna().sum()}")

# Fill NaN values with empty string to avoid errors
bmg_drug_name_df['Drug_Name'] = bmg_drug_name_df['Drug_Name'].fillna('')

# Convert to string to ensure split() works on all entries
bmg_drug_name_df['drug_name_list'] = bmg_drug_name_df['Drug_Name'].astype(str).apply(lambda x: [name.strip() for name in x.split(';')] if x else [])

# Similarly for drug_name_df
drug_name_df['drug_name_list'] = drug_name_df['DRUG_NAME'].astype(str).apply(lambda x: [name.strip() for name in x.split(';')] if x else [])

In [ ]:
display(bmg_drug_name_df)
display(drug_name_df)

In [ ]:
# Create a flattened mapping of drug names to their BMG IDs
drug_name_to_bmg = {}
for idx, row in bmg_drug_name_df.iterrows():
    bmg_id = row['BioMedGraphica_Conn_ID']
    for drug_name in row['drug_name_list']:
        if drug_name:  # Avoid empty strings
            drug_name_to_bmg[drug_name.upper()] = bmg_id

# Create a new mapping from ARXSPAN_ID to BMG_ID based on drug name matches
arxspan_to_bmg = {}
for idx, row in drug_name_df.iterrows():
    arxspan_id = row['DRUG_NAME']
    for drug_name in row['drug_name_list']:
        if drug_name.upper() in drug_name_to_bmg:
            arxspan_to_bmg[arxspan_id] = drug_name_to_bmg[drug_name.upper()]
            break

# Create a mapping dataframe
mapping_df = pd.DataFrame(list(arxspan_to_bmg.items()), columns=['DRUG_NAME', 'BioMedGraphica_Conn_ID'])

# Display how many drug names were successfully matched
print(f"Successfully matched {len(mapping_df)} out of {len(drug_name_df)} drugs")

# Display the first few rows of the mapping
print("\nSample of drug name mappings:")
display(mapping_df)

# Now you can use this mapping to merge with your drug_response_df
# Example:
# merged_df = pd.merge(drug_response_df, mapping_df, on='DRUG_NAME', how='left')


In [ ]:
final_merged_drug_df = mapping_df.copy()
# sort values by the column BioMedGraphica_ID
final_merged_drug_df = final_merged_drug_df.sort_values(by='BioMedGraphica_Conn_ID').reset_index(drop=True)
display(final_merged_drug_df)

In [ ]:
# filter the final drug score dataframe by filtering the 'DRUG_NAME' in the final_merged_drug_df DRUG_NAME
final_drug_df = raw_drug_df[raw_drug_df['DRUG_NAME'].isin(final_merged_drug_df['DRUG_NAME'])].reset_index(drop=True)
display(final_drug_df)

## 6. Coalign the samples

In [ ]:
methy_samples = final_merged_methy_df.columns[1:]
gene_samples = final_merged_gene_df.columns[1:]
transcript_samples = final_merged_transcript_df.columns[1:]
protein_samples = final_merged_protein_df.columns[1:]
drug_samples = list(set(list(final_drug_df['ARXSPAN_ID'])))

# print all samples
print(methy_samples)
print(gene_samples)
print(transcript_samples)
print(protein_samples)
print(drug_samples)

In [ ]:
# cell line names in the protein samples, remove the content after the second "_"
# result = ['_'.join(s.split('_')[:2]) for s in strings]
protein_split_samples = ['_'.join(s.split('_')[:2]) for s in protein_samples]
print(protein_split_samples)

In [ ]:
cell_line_anno_df = pd.read_csv('./raw_data/Cell_lines_annotations_20181226.txt', sep='\t')
# drop NaN in the depMapID
cell_line_anno_df = cell_line_anno_df.dropna(subset=['depMapID']).reset_index(drop=True)
# drop NaN in the Pathology
cell_line_anno_df = cell_line_anno_df.dropna(subset=['Pathology']).reset_index(drop=True)
# drop rows if the column 'PATHOLOGIST_ANNOTATION' contains 'benign'
# First handle NaN values in PATHOLOGIST_ANNOTATION
cell_line_anno_df = cell_line_anno_df[~(
    cell_line_anno_df['PATHOLOGIST_ANNOTATION'].fillna('').str.contains('benign', case=False)
)]
display(cell_line_anno_df)

In [ ]:
print(len(list(set(protein_split_samples))))
# give me the duplicate elements in the protein_split_samples
import collections
rep_list = [item for item, count in collections.Counter(protein_split_samples).items() if count > 1]
# for column names in final_merged_protein_df, if the column name contains the element in the rep_list, collect the column name in rep_col_list
rep_col_list = [col for col in final_merged_protein_df.columns if any(rep in col for rep in rep_list)]
print(rep_col_list)
# display the rep_col_list in the final_merged_protein_df
display(final_merged_protein_df[rep_col_list])

# removed all of the rep_col_list in the final_merged_protein_df
final_merged_protein_df = final_merged_protein_df.drop(columns=rep_col_list)
display(final_merged_protein_df)

In [ ]:
# get the overlapped cell lines in methy_samples with cell_line_anno_df by merge on cell line id
# first format the methy_samples as dataframe
methy_samples_df = pd.DataFrame(methy_samples, columns=['CCLE_Name'])
# merge the methy_samples_df with the cell_line_anno_df
merged_methy_samples_df = pd.merge(methy_samples_df, cell_line_anno_df, left_on='CCLE_Name', right_on='CCLE_ID', how='inner')
display(merged_methy_samples_df)

# get the map dictionary from the ccle id to depmap id
methy_map_dict = dict(zip(merged_methy_samples_df['CCLE_Name'], merged_methy_samples_df['depMapID']))
print(methy_map_dict)

In [ ]:
# get the cleaned cell line names by removed the content after the second "_" in final_merged_protein_df.columns[1:]
original_protein_samples = final_merged_protein_df.columns[1:]
cleaned_protein_samples = ['_'.join(s.split('_')[:2]) for s in original_protein_samples]
# get the cleaned_protein_samples_dict_df from dictionary
cleaned_protein_samples_dict_df = pd.DataFrame({'CCLE_Name': original_protein_samples, 'cleaned_CCLE_Name': cleaned_protein_samples})
# merge the cleaned_protein_samples_df with the cell_line_anno_df
merged_cleaned_protein_samples_df = pd.merge(cleaned_protein_samples_dict_df, cell_line_anno_df, left_on='cleaned_CCLE_Name', right_on='CCLE_ID', how='inner')
display(merged_cleaned_protein_samples_df)
# get the depmap protein id
depmap_protein_samples = merged_cleaned_protein_samples_df['depMapID'].to_list()

# Create the map dictionary for the protein samples
protein_map_dict = dict(zip(original_protein_samples, depmap_protein_samples))
print(protein_map_dict)

In [ ]:
# get the methy_mapped_samples and protein_mapped_samples from merged_methy_samples_df and merged_cleaned_protein_samples_df
methy_mapped_samples = list(merged_methy_samples_df['depMapID'])
protein_mapped_samples = list(merged_cleaned_protein_samples_df['depMapID'])

### 6.1 Aligning over omics

#### 6.1.1 Aligning with intersection

In [ ]:
# get the overlapped samples only over omics samples
omics_overlapped_samples = sorted(list(set(methy_mapped_samples) & set(gene_samples) & set(transcript_samples) & set(protein_mapped_samples)))
print("length of omics overlapped samples: ", len(omics_overlapped_samples))
# check the overlapped samples with annotation samples
annotation_samples = list(cell_line_anno_df['depMapID'])
overlapped_omics_annotation_samples = sorted(list(set(omics_overlapped_samples) & set(annotation_samples)))
print("length of overlapped omics and annotation samples: ", len(overlapped_omics_annotation_samples))

#### 6.1.2 Aligning with union on omics and intersection with annotation

In [ ]:
# get the union samples only over omics samples
omics_union_samples = sorted(list(set(methy_mapped_samples) | set(gene_samples) | set(transcript_samples) | set(protein_mapped_samples)))
print("length of omics union samples: ", len(omics_union_samples))
# check the intersection of union samples with annotation samples
overlapped_omics_union_annotation_samples = sorted(list(set(omics_union_samples) & set(annotation_samples)))
print("length of overlapped omics and annotation samples: ", len(overlapped_omics_union_annotation_samples))
print(overlapped_omics_union_annotation_samples)

#### 6.1.3 overlapped_omics_union_annotation_samples (cancerous / non-cancerous) Ratio

In [ ]:
non_cancerous_samples_df = pd.read_csv('./raw_data/cell-lines-in-Non-Cancerous.csv')
display(non_cancerous_samples_df)
# get the depmap id from the non_cancerous_samples_df
non_cancerous_samples = list(non_cancerous_samples_df['Depmap Id'])
# get the overlapped non_cancerous samples with the overlapped_omics_union_annotation_samples
overlapped_non_cancerous_samples = sorted(list(set(overlapped_omics_union_annotation_samples) & set(non_cancerous_samples)))
print("length of overlapped omics and annotation samples: ", len(overlapped_omics_union_annotation_samples))
print("length of overlapped non cancerous samples: ", len(overlapped_non_cancerous_samples))


### 6.2 Overlapping over DTI

In [ ]:
# get the overlapped samples among all samples (methy_mapped_samples, gene_samples, transcript_samples, protein_mapped_samples, drug_samples)
dti_overlapped_samples = sorted(list(set(overlapped_omics_union_annotation_samples) & set(drug_samples)))
print("len(overlapped_samples):", len(dti_overlapped_samples))
# need to confirm zero intersection between the overlapped samples and non_cancerous_samples
dti_overlapped_non_cancerous_samples = sorted(list(set(dti_overlapped_samples) & set(non_cancerous_samples)))
print("len(overlapped_non_cancerous_samples):", len(dti_overlapped_non_cancerous_samples))
# if there are overlapped samples between the dti_overlapped_samples and non_cancerous_samples, then remove these samples from the dti_overlapped_samples
if len(dti_overlapped_non_cancerous_samples) > 0:
    dti_overlapped_samples = sorted(list(set(dti_overlapped_samples) - set(dti_overlapped_non_cancerous_samples)))
# convert the overlapped_samples to the dataframe with annotation
dti_overlapped_samples_df = pd.merge(pd.DataFrame(dti_overlapped_samples, columns=['depMapID']), cell_line_anno_df, on='depMapID', how='inner')
display(dti_overlapped_samples_df)

In [ ]:
# mkdir for processsing folder
import os
if not os.path.exists('./process_data'):
    os.makedirs('./process_data')

In [ ]:
dti_overlapped_samples_df.to_csv('./process_data/dti_overlapped_samples.csv', index=False)

In [ ]:
print(dti_overlapped_samples_df.columns)

## 7. Annotate the samples with disease

### 7.1 Add annotation from Cellosaurus

In [ ]:
import requests

url = 'https://ftp.expasy.org/databases/cellosaurus/cellosaurus.obo'
response = requests.get(url)

with open('./raw_data/cellosaurus.obo', 'wb') as f:
    f.write(response.content)

In [ ]:
import pandas as pd

# Path to the downloaded OBO file
obo_path = './raw_data/cellosaurus.obo'

entries = []
current_entry = {}

# Read and parse
with open(obo_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()

        if line == "[Term]":
            if current_entry.get("xref") and any("NCBI_TaxID:9606" in x for x in current_entry["xref"]):
                # Separate NCIt and ORDO xrefs
                ncit_refs = [x for x in current_entry["xref"] if x.startswith("NCIt:")]
                ordo_refs = [x for x in current_entry["xref"] if x.startswith("ORDO:")]
                current_entry["xref_NCIt"] = "; ".join(ncit_refs)
                current_entry["xref_ORDO"] = "; ".join(ordo_refs)
                entries.append({
                    "id": current_entry.get("id", ""),
                    "name": current_entry.get("name", ""),
                    "synonym": "; ".join(current_entry.get("synonym", [])),
                    "xref_NCIt": current_entry.get("xref_NCIt", ""),
                    "xref_ORDO": current_entry.get("xref_ORDO", "")
                })
            current_entry = {"synonym": [], "xref": []}

        elif line.startswith("id:"):
            current_entry["id"] = line.split("id:")[1].strip()

        elif line.startswith("name:"):
            current_entry["name"] = line.split("name:")[1].strip()

        elif line.startswith("synonym:"):
            synonym = line.split("synonym:")[1].split("RELATED")[0].strip().strip('"')
            current_entry["synonym"].append(synonym)

        elif line.startswith("xref:"):
            current_entry["xref"].append(line.split("xref:")[1].strip())

# Handle last entry
if current_entry.get("xref") and any("NCBI_TaxID:9606" in x for x in current_entry["xref"]):
    ncit_refs = [x for x in current_entry["xref"] if x.startswith("NCIt:")]
    ordo_refs = [x for x in current_entry["xref"] if x.startswith("ORDO:")]
    entries.append({
        "id": current_entry.get("id", ""),
        "name": current_entry.get("name", ""),
        "synonym": "; ".join(current_entry.get("synonym", [])),
        "xref_NCIt": "; ".join(ncit_refs),
        "xref_ORDO": "; ".join(ordo_refs)
    })

# Create DataFrame
cellosaurus_parsed_df = pd.DataFrame(entries)
cellosaurus_parsed_df = cellosaurus_parsed_df[["id", "name", "synonym", "xref_NCIt", "xref_ORDO"]]
cellosaurus_parsed_df.to_csv('./raw_data/cellosaurus_parsed.csv', index=False)
# Show preview
display(cellosaurus_parsed_df)

In [ ]:
# Step 1: Keep the desired columns
dti_overlapped_samples_desc_df = dti_overlapped_samples_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code']]
display(dti_overlapped_samples_desc_df)

# Step 2: Try direct merge on Name == name using left join
matched_df = pd.merge(
    dti_overlapped_samples_desc_df,
    cellosaurus_parsed_df,
    left_on='Name',
    right_on='name',
    how='left'
)

# Step 3: Find rows where no match occurred (name is NaN)
unmatched_df = matched_df[matched_df['name'].isna()].copy()

# Step 4: Drop the Cellosaurus columns to prep for synonym match
cols_to_drop = [col for col in matched_df.columns if col in cellosaurus_parsed_df.columns]
unmatched_df = unmatched_df.drop(columns=cols_to_drop)

# Step 5: Expand cellosaurus synonyms
synonym_expanded_df = cellosaurus_parsed_df.copy()
synonym_expanded_df = synonym_expanded_df.dropna(subset=['synonym'])
synonym_expanded_df = synonym_expanded_df.assign(
    synonym=synonym_expanded_df['synonym'].str.split(';')
).explode('synonym')
synonym_expanded_df['synonym'] = synonym_expanded_df['synonym'].str.strip()

# Step 6: Left join unmatched rows with synonym-expanded Cellosaurus
synonym_matched_df = pd.merge(
    unmatched_df,
    synonym_expanded_df,
    left_on='Name',
    right_on='synonym',
    how='left'
)

# Step 7: Combine direct-matched (non-NaN name) and synonym-matched rows
all_match_df = pd.concat(
    [matched_df[matched_df['name'].notna()], synonym_matched_df],
    ignore_index=True
)

# Optional: Drop duplicates based on depMapID if needed
all_match_df = all_match_df.drop_duplicates(subset=['depMapID'])
display(all_match_df)

In [ ]:
# Function to store primary and extra xref matches with better column handling
def process_xref_columns_with_extras(df):
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Always create all the columns, even if they might end up empty
    result_df['xref_NCIt_ID'] = ''
    result_df['xref_NCIt_name'] = ''
    result_df['xref_NCIt_extra_ID'] = ''
    result_df['xref_NCIt_extra_name'] = ''
    result_df['xref_ORDO_ID'] = ''
    result_df['xref_ORDO_name'] = ''
    result_df['xref_ORDO_extra_ID'] = ''
    result_df['xref_ORDO_extra_name'] = ''
    
    # Process xref_NCIt if it exists
    if 'xref_NCIt' in result_df.columns:
        # Only process rows that have xref_NCIt values
        mask = result_df['xref_NCIt'].notna() & (result_df['xref_NCIt'] != '')
        
        for idx in result_df[mask].index:
            xref_value = result_df.loc[idx, 'xref_NCIt']
            
            if '; ' in xref_value:  # Multiple entries
                entries = xref_value.split('; ')
                
                # Process first entry for primary columns
                if ' ! ' in entries[0]:
                    id_val, name_val = entries[0].split(' ! ', 1)
                    result_df.loc[idx, 'xref_NCIt_ID'] = id_val
                    result_df.loc[idx, 'xref_NCIt_name'] = name_val
                
                # Process additional entries for extra columns
                extra_ids = []
                extra_names = []
                for entry in entries[1:]:
                    if ' ! ' in entry:
                        id_val, name_val = entry.split(' ! ', 1)
                        extra_ids.append(id_val)
                        extra_names.append(name_val)
                
                if extra_ids:
                    result_df.loc[idx, 'xref_NCIt_extra_ID'] = '; '.join(extra_ids)
                    result_df.loc[idx, 'xref_NCIt_extra_name'] = '; '.join(extra_names)
                    
            elif ' ! ' in xref_value:  # Single entry
                id_val, name_val = xref_value.split(' ! ', 1)
                result_df.loc[idx, 'xref_NCIt_ID'] = id_val
                result_df.loc[idx, 'xref_NCIt_name'] = name_val
        
        # Drop the original column
        result_df = result_df.drop(columns=['xref_NCIt'])
    
    # Process xref_ORDO if it exists
    if 'xref_ORDO' in result_df.columns:
        # Only process rows that have xref_ORDO values
        mask = result_df['xref_ORDO'].notna() & (result_df['xref_ORDO'] != '')
        
        for idx in result_df[mask].index:
            xref_value = result_df.loc[idx, 'xref_ORDO']
            
            if '; ' in xref_value:  # Multiple entries
                entries = xref_value.split('; ')
                
                # Process first entry for primary columns
                if ' ! ' in entries[0]:
                    id_val, name_val = entries[0].split(' ! ', 1)
                    result_df.loc[idx, 'xref_ORDO_ID'] = id_val
                    result_df.loc[idx, 'xref_ORDO_name'] = name_val
                
                # Process additional entries for extra columns
                extra_ids = []
                extra_names = []
                for entry in entries[1:]:
                    if ' ! ' in entry:
                        id_val, name_val = entry.split(' ! ', 1)
                        extra_ids.append(id_val)
                        extra_names.append(name_val)
                
                if extra_ids:
                    result_df.loc[idx, 'xref_ORDO_extra_ID'] = '; '.join(extra_ids)
                    result_df.loc[idx, 'xref_ORDO_extra_name'] = '; '.join(extra_names)
                    
            elif ' ! ' in xref_value:  # Single entry
                id_val, name_val = xref_value.split(' ! ', 1)
                result_df.loc[idx, 'xref_ORDO_ID'] = id_val
                result_df.loc[idx, 'xref_ORDO_name'] = name_val
        
        # Drop the original column
        result_df = result_df.drop(columns=['xref_ORDO'])
    
    return result_df

# Apply the function
all_match_df = process_xref_columns_with_extras(all_match_df)

# Update the column list to include the extra columns
dti_overlapped_samples_desc_co_df = all_match_df[['depMapID', 'Name', 'Pathology', 'Histology', 
                                             'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 
                                             'xref_NCIt_ID', 'xref_NCIt_name', 'xref_NCIt_extra_ID', 'xref_NCIt_extra_name',
                                             'xref_ORDO_ID', 'xref_ORDO_name', 'xref_ORDO_extra_ID', 'xref_ORDO_extra_name']]
display(dti_overlapped_samples_desc_co_df)

### 7.2 Cell line disease match

#### 7.2.1 Cell line disease hard match

In [ ]:
bmg_disease_name_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Disease/BioMedGraphica_Conn_Disease_GUI_Name.csv')
display(bmg_disease_name_df)

In [ ]:
# Step 1: Expand BMG_Disease_Name (still lowercased and exploded)
bmg_expanded = bmg_disease_name_df.copy()

# First, strip any surrounding quotes and fill NaN values with empty strings
bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].fillna('')
bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].str.replace('^"|"$', '', regex=True)

# Split on pipe with spaces
bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].str.split(' \| ')

# Apply strip only to non-empty lists
bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].apply(
    lambda x: [name.strip() for name in x] if isinstance(x, list) else []
)

# Explode the list into separate rows
bmg_expanded = bmg_expanded.explode('Disease_Name_List')

# Remove rows with empty disease names after exploding
bmg_expanded = bmg_expanded[bmg_expanded['Disease_Name_List'].str.len() > 0]

# Create lowercase version for easier matching
bmg_expanded['Disease_Name_List_lower'] = bmg_expanded['Disease_Name_List'].str.lower()

display(bmg_expanded)

In [ ]:
# Create a lookup list of tuples: (Conn_ID, Original_Name, Lower_Name)
bmg_records = bmg_expanded[['BioMedGraphica_Conn_ID', 'Disease_Name_List', 'Disease_Name_List_lower']].to_records(index=False)

# Updated matching function: look for exact full-name match
def match_bmg_disease_exact(xref_name, bmg_records):
    if pd.isna(xref_name):
        return (None, None)
    xref_name_lower = str(xref_name).strip().lower()
    
    for conn_id, original_name, bmg_name_lower in bmg_records:
        if xref_name_lower == bmg_name_lower:
            return (conn_id, original_name)
    
    return (None, None)

# NCIt
ncit_matches = dti_overlapped_samples_desc_co_df['xref_NCIt_name'].apply(lambda x: match_bmg_disease_exact(x, bmg_records))
dti_overlapped_samples_desc_co_df['NCIt_BMGC_ID'] = ncit_matches.apply(lambda x: x[0])
dti_overlapped_samples_desc_co_df['NCIt_BMGC_name'] = ncit_matches.apply(lambda x: x[1])

# ORDO
ordo_matches = dti_overlapped_samples_desc_co_df['xref_ORDO_name'].apply(lambda x: match_bmg_disease_exact(x, bmg_records))
dti_overlapped_samples_desc_co_df['ORDO_BMGC_ID'] = ordo_matches.apply(lambda x: x[0])
dti_overlapped_samples_desc_co_df['ORDO_BMGC_name'] = ordo_matches.apply(lambda x: x[1])

display(dti_overlapped_samples_desc_co_df)


In [ ]:
# keep the columns ['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_ID', 'xref_NCIt_name', 'NCIt_BMGC_ID', 'NCIt_BMGC_name', 'xref_ORDO_ID', 'xref_ORDO_name', 'ORDO_BMGC_ID', 'ORDO_BMGC_name']
dti_overlapped_samples_desc_co_df = dti_overlapped_samples_desc_co_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_ID', 'xref_NCIt_name', 'NCIt_BMGC_ID', 'NCIt_BMGC_name', 'xref_ORDO_ID', 'xref_ORDO_name', 'ORDO_BMGC_ID', 'ORDO_BMGC_name']]
display(dti_overlapped_samples_desc_co_df)
dti_overlapped_samples_desc_co_df.to_csv('./process_data/dti_overlapped_samples_desc_co.csv', index=False)

In [ ]:
# Merge NCIt_BMGC_ID/ORDO_BMGC_ID and NCIt_BMGC_name/ORDO_BMGC_name into BMGC_Matched columns
# Strategy:
# 1. Prioritize NCIt data when both are available
# 2. Use NCIt data if only NCIt is available
# 3. Use ORDO data if only ORDO is available
# 4. Leave as null if both are null

dti_matched_df = dti_overlapped_samples_desc_co_df.copy()

# Create new merged ID column
dti_matched_df['BMGC_Matched_ID'] = (
    dti_matched_df['NCIt_BMGC_ID'].fillna(
        dti_matched_df['ORDO_BMGC_ID']
    )
)

# Create new merged name column
dti_matched_df['BMGC_Matched_name'] = (
    dti_matched_df['NCIt_BMGC_name'].fillna(
        dti_matched_df['ORDO_BMGC_name']
    )
)

# Display the dataframe with new columns
display(dti_matched_df)

# Check number of null values in new columns
print(f"Null values in BMGC_Matched_ID: {dti_matched_df['BMGC_Matched_ID'].isna().sum()}")
print(f"Null values in BMGC_Matched_name: {dti_matched_df['BMGC_Matched_name'].isna().sum()}")

# only keep the columns where the BMGC_Matched_ID is not null
dti_matched_df = dti_matched_df[dti_matched_df['BMGC_Matched_ID'].notna()].reset_index(drop=True)
display(dti_matched_df)

# Save to CSV
dti_matched_df.to_csv('./process_data/dti_matched.csv', index=False)

#### 7.2.2 Cell line disease soft match

In [ ]:
# check the unmatched samples in the dti_overlapped_samples_desc_co_df by checking the NaN in the NCIt_BMGC_ID and ORDO_BMGC_ID
unmatched_samples_df = dti_overlapped_samples_desc_co_df[dti_overlapped_samples_desc_co_df['NCIt_BMGC_ID'].isna() & dti_overlapped_samples_desc_co_df['ORDO_BMGC_ID'].isna()]
# keep the columns ['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_name', , 'xref_ORDO_name']
unmatched_samples_df = unmatched_samples_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_name', 'xref_ORDO_name']]
display(unmatched_samples_df)

In [ ]:
# embed the disease name for bmg_disease_df by bert-based model
import torch
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset

class SentenceDataset(Dataset):
    def __init__(self, sentences: List[str]):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]


class TextEncoder():
    def __init__(self, model_path: str = "dmis-lab/biobert-v1.1", device: str = "cuda"):
        """
        Args:
            model_path (str, optional): Path to the deberta model. Defaults to 'dmis-lab/biobert-v1.1'.
            device (str, optional): Device to run the model on ('cpu' or 'cuda'). Defaults to 'cpu'.
        """
        self.model_path = model_path
        self.device = device
        self.model = None
        self.tokenizer = None

    def load_model(self):
        """
        Load the deberta model and tokenizer from the specified model path.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path).to(self.device)

    def generate_embeddings(self, sentences: List[str], batch_size: int = 32, seq_emb_dim: int = 64) -> torch.Tensor:
        """
        Generate a single-dimensional embedding for each sentence.

        Args:
            sentences (List[str]): List of sentences to embed.
            batch_size (int, optional): Batch size for DataLoader. Defaults to 32.

        Returns:
            List[float]: List of single-dimensional embeddings.
        """
        dataset = SentenceDataset(sentences)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        embedding_batches = []
        for batch in tqdm(dataloader, desc="Embedding sentences", unit="batch"):
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Handle single batch case properly
            mean_embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # [batch_size, hidden_dim]
            
            # For adaptive pooling, we need to reshape for 1D adaptive pooling
            # [batch_size, 1, hidden_dim] -> [batch_size, 1, seq_emb_dim] -> [batch_size, seq_emb_dim]
            batch_size = mean_embeddings.size(0)
            reshaped = mean_embeddings.view(batch_size, 1, -1)
            projected = torch.nn.functional.adaptive_avg_pool1d(reshaped, output_size=seq_emb_dim)
            projected = projected.squeeze(1)  # Only squeeze dimension 1, keep batch dimension
            embedding_batches.append(projected)
        return torch.cat(embedding_batches, dim=0)

    def save_embeddings(self, embeddings, output_npy_path):
        """
        Save embeddings to a .npy file.
        
        Args:
            embeddings (torch.Tensor): The embeddings to save.
            output_npy_path (str): Path to save the embeddings file.
        """
        # Move embeddings to CPU before converting to numpy
        embeddings_cpu = embeddings.cpu().numpy()
        np.save(output_npy_path, embeddings_cpu)
        print(f"Embeddings saved at {output_npy_path} with shape {embeddings_cpu.shape}")

In [ ]:
# convert the bmg_disease_name_df to the list of disease names of each rows
disease_names = list(bmg_disease_name_df['Disease_Name_List'])
# to make sure each disease name is a string
disease_names = [str(name) for name in disease_names]
print(len(disease_names))
print(disease_names[:5])
print(disease_names[-1])

In [ ]:
 # Use language model to embed the name and description
name_sentence_list = disease_names
text_encoder = TextEncoder()
text_encoder.load_model()
name_embeddings = text_encoder.generate_embeddings(name_sentence_list, batch_size=32, seq_emb_dim=768)
print(name_embeddings.shape)
# mkdir folder BMG_emb
if not os.path.exists('./BMG_emb'):
    os.makedirs('./BMG_emb')
text_encoder.save_embeddings(name_embeddings, './BMG_emb/disease_name_embeddings.npy')

In [ ]:
# convert each row in the unmatched_samples_df to a sentence like "Name: NIH:OVCAR-3, Pathology: OVARY, Histology: CARCINOMA, type: CANCER, PATHOLOGIST_ANNOTATION: OVARY, tcga_code: OV"
depmap_desc_sentence_list = []
for idx, row in unmatched_samples_df.iterrows():
    depmap_desc_sentence_list.append(f" {row['type']}, {row['PATHOLOGIST_ANNOTATION']}")

# and create the ncit_desc_sentence_list and ordo_desc_sentence_list
ncit_desc_sentence_list = list(unmatched_samples_df['xref_NCIt_name'])
ordo_desc_sentence_list = list(unmatched_samples_df['xref_ORDO_name'])

# make sure each element in the depmap_desc_sentence_list, ncit_desc_sentence_list and ordo_desc_sentence_list is a string
depmap_desc_sentence_list = [str(name) for name in depmap_desc_sentence_list]
ncit_desc_sentence_list = [str(name) for name in ncit_desc_sentence_list]
ordo_desc_sentence_list = [str(name) for name in ordo_desc_sentence_list]

print(depmap_desc_sentence_list)
print(ncit_desc_sentence_list)
print(ordo_desc_sentence_list)

In [ ]:
# Define the sentence lists to process and their corresponding prefixes
sentence_list_mapping = {
    'depmap': depmap_desc_sentence_list,
    'ncit': ncit_desc_sentence_list,
    'ordo': ordo_desc_sentence_list
}

# Define top_k for this matching task
top_k = 2

# Process each sentence list and add the matches to the DataFrame
for prefix, sentence_list in sentence_list_mapping.items():
    # Lists to store the matches for each description
    all_matched_lists = [[] for _ in range(top_k)]
    all_matched_name_lists = [[] for _ in range(top_k)]
    all_similarity_lists = [[] for _ in range(top_k)]
    
    for desc in sentence_list:
        desc_embeddings = text_encoder.generate_embeddings([desc], batch_size=1, seq_emb_dim=768)
        # Calculate cosine similarity
        cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
        similarity = cos(name_embeddings, desc_embeddings)
        # Find the index of top k most similar diseases
        top_k_idx = torch.argsort(similarity, descending=True)[:top_k]
        
        # Process each of the top k matches
        for rank, idx in enumerate(top_k_idx):
            idx_int = idx.item()
            disease_id = bmg_disease_name_df.iloc[idx_int]['BioMedGraphica_Conn_ID']
            disease_name = disease_names[idx_int]
            sim_score = similarity[idx_int].item()
            
            # Store in the corresponding lists
            all_matched_lists[rank].append(disease_id)
            all_matched_name_lists[rank].append(disease_name)
            all_similarity_lists[rank].append(sim_score)
    
    # Add the matched data to the DataFrame with the appropriate prefix
    for i in range(top_k):
        rank = i + 1
        unmatched_samples_df[f'{prefix}_match_{rank}_disease'] = all_matched_lists[i]
        unmatched_samples_df[f'{prefix}_match_{rank}_disease_name'] = all_matched_name_lists[i]
        unmatched_samples_df[f'{prefix}_match_{rank}_similarity'] = all_similarity_lists[i]
    
    print(f"Finished processing {prefix} descriptions")

# Display the updated DataFrame
display(unmatched_samples_df)

# Save the updated DataFrame to CSV
unmatched_samples_df.to_csv('./process_data/unmatched_samples_softmatch.csv', index=False)

### 7.3 Manual filter and match

In [ ]:
unmatched_samples_df = pd.read_csv('./process_data/unmatched_samples_softmatch.csv')
# add the 2 empty columnns 'BMGC_manual_ID' and 'BMGC_manual_name' to the unmatched_samples_manual_df
unmatched_samples_manual_df = unmatched_samples_df.copy()
unmatched_samples_manual_df['BMGC_manual_ID'] = None
unmatched_samples_manual_df['BMGC_manual_name'] = None
# change the order of the columns to make the 'BMGC_manual_ID' and 'BMGC_manual_name' after id
unmatched_samples_manual_df = unmatched_samples_manual_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'BMGC_manual_ID', 'BMGC_manual_name', 'xref_NCIt_name', 'xref_ORDO_name', 'depmap_match_1_disease', 'depmap_match_1_disease_name', 'depmap_match_2_disease', 'depmap_match_2_disease_name', 'ncit_match_1_disease', 'ncit_match_1_disease_name', 'ncit_match_2_disease', 'ncit_match_2_disease_name', 'ordo_match_1_disease', 'ordo_match_1_disease_name', 'ordo_match_2_disease', 'ordo_match_2_disease_name']]
display(unmatched_samples_manual_df)
unmatched_samples_manual_df.to_csv('./process_data/unmatched_samples_manual.csv', index=False)

### 7.4 Combine matched and manual_unmatached

In [ ]:
# Matched samples, rename BMGC_Matched_ID and BMGC_Matched_name to BMGC_ID and BMGC_name
dti_matched_df = pd.read_csv('./process_data/dti_matched.csv').rename(columns={'BMGC_Matched_ID': 'BMGC_Disease_ID', 'BMGC_Matched_name': 'BMGC_Disease_name'})
# keep the columns ['depMapID', 'Name', 'BMGC_Disease_ID', 'BMGC_Disease_name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'xref_NCIt_ID', 'xref_NCIt_name', 'xref_ORDO_ID', 'xref_ORDO_name']
dti_matched_df = dti_matched_df[['depMapID', 'Name', 'BMGC_Disease_ID', 'BMGC_Disease_name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code']]
display(dti_matched_df)
# Unmatched samples, rename BMGC_manual_ID and BMGC_manual_name to BMGC_Disease_ID and BMGC_Disease_name
dti_unmatched_manual_df = pd.read_csv('./process_data/unmatched_samples_manual.csv').rename(columns={'BMGC_manual_ID': 'BMGC_Disease_ID', 'BMGC_manual_name': 'BMGC_Disease_name'})
display(dti_unmatched_manual_df)

In [ ]:
dti_unmatched_manual_valid_df = pd.read_csv('./process_data/unmatched_samples_manual_valid.csv').rename(columns={'BMGC_manual_ID': 'BMGC_Disease_ID', 'BMGC_manual_name': 'BMGC_Disease_name'})
display(dti_unmatched_manual_valid_df)
# concatenate the dti_matched_df and dti_unmatched_manual_df and sort by depMapID
dti_combined_df = pd.concat([dti_matched_df, dti_unmatched_manual_valid_df], ignore_index=True)
# sort the dti_combined_df by depMapID
dti_combined_df = dti_combined_df.sort_values(by='depMapID').reset_index(drop=True)
# check if there is any empty value in the BMGC_ID and BMGC_name columns
print(dti_combined_df[['BMGC_Disease_ID', 'BMGC_Disease_name']].isnull().sum())
# convert all content in 'BMGC_Disease_name' to lower case
dti_combined_df['BMGC_Disease_name'] = dti_combined_df['BMGC_Disease_name'].str.lower()
display(dti_combined_df)
dti_combined_df.to_csv('./process_data/dti_combined_samples.csv', index=False)

## 8. Split train and test

### 8.1 DTI integration

In [ ]:
dti_overlapped_samples = list(dti_overlapped_samples_df['depMapID'])
print(dti_overlapped_samples)

In [ ]:
maped_methy_df = final_merged_methy_df.rename(columns=methy_map_dict)
display(maped_methy_df)
# insert the samples that is in the overlapped_samples but not in the maped_methy_df
for sample in dti_overlapped_samples:
    if sample not in maped_methy_df.columns:
        maped_methy_df[sample] = 0.0
# filter out the samples that is not in the dti_overlapped_samples
dti_methy_df = maped_methy_df[['BioMedGraphica_Conn_ID'] + dti_overlapped_samples]
display(dti_methy_df)

In [ ]:
mapped_protein_df = final_merged_protein_df.rename(columns=protein_map_dict)
display(mapped_protein_df)
# insert the samples that is in the dti_overlapped_samples but not in the mapped_protein_df
for sample in dti_overlapped_samples:
    if sample not in mapped_protein_df.columns:
        mapped_protein_df[sample] = 0.0
# filter out the samples that is not in the dti_overlapped_samples
dti_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + dti_overlapped_samples]
display(dti_protein_df)

In [ ]:
# fill in the missing samples in the final_merged_gene_df
for sample in dti_overlapped_samples:
    if sample not in final_merged_gene_df.columns:
        final_merged_gene_df[sample] = 0.0
# fill in the missing samples in the final_merged_transcript_df
for sample in dti_overlapped_samples:
    if sample not in final_merged_transcript_df.columns:
        final_merged_transcript_df[sample] = 0.0
# get the final gene, transcript, drug dataframe by filtering the dti_overlapped_samples
dti_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + dti_overlapped_samples]
display(dti_gene_df)
dti_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + dti_overlapped_samples]
display(dti_transcript_df)
dti_drug_overlap_df = final_drug_df[final_drug_df['ARXSPAN_ID'].isin(dti_overlapped_samples)].reset_index(drop=True)
dti_drug_overlap_df = pd.merge(dti_drug_overlap_df, dti_combined_df, left_on='ARXSPAN_ID', right_on='depMapID', how='left')
dti_drug_overlap_df = pd.merge(dti_drug_overlap_df, final_merged_drug_df, on='DRUG_NAME', how='left').rename(columns={'BMGC_Disease_name': 'BMGC_Disease_Name', 'DRUG_NAME': 'BMGC_Drug_Name', 'BioMedGraphica_Conn_ID': 'BMGC_Drug_ID', 'Name': 'Cell_Line_Name', 'AUC_PUBLISHED': 'AUC'})
# only keep columns ['depMapID', 'Cell_Line_Name', 'BMGC_Drug_ID', 'BMGC_Drug_Name', 'BMGC_Disease_ID', 'BMGC_Disease_Name', 'AUC']
dti_drug_overlap_df = dti_drug_overlap_df[['depMapID', 'Cell_Line_Name', 'BMGC_Drug_ID', 'BMGC_Drug_Name', 'BMGC_Disease_ID', 'BMGC_Disease_Name', 'AUC']]
# check if there is null values in the dti_drug_overlap_df
print(dti_drug_overlap_df.isnull().sum())
display(dti_drug_overlap_df)

In [ ]:
# check if there is any empty or Null values in the dti_drug_overlap_df
print(dti_drug_overlap_df.isnull().sum())

In [ ]:
dti_omics_df = pd.concat([dti_methy_df, dti_gene_df, dti_transcript_df, dti_protein_df], axis=0).reset_index(drop=True)
display(dti_omics_df)

### 8.2 DTI Split the samples for training and test

In [ ]:
dti_samples =  dti_combined_df['depMapID'].tolist()
print(dti_samples)

In [ ]:
# split the overlapped samples into training samples and test samples
ratio = 0.8
import numpy as np
np.random.seed(0)
# shuffle the overlapped_samples
np.random.shuffle(dti_samples)
# get the training samples and test samples
dti_training_samples = dti_samples[:int(len(dti_samples) * ratio)]
dti_test_samples = dti_samples[int(len(dti_samples) * ratio):]
print("len(dti_training_samples):", len(dti_training_samples))
print("len(dti_test_samples):", len(dti_test_samples))

In [ ]:
# split the drugs into training samples and test samples
np.random.seed(0)
# shuffle the drug_samples
display(final_merged_drug_df)
dti_drugs = final_merged_drug_df['BioMedGraphica_Conn_ID'].to_list()
np.random.shuffle(dti_drugs)
# get the training samples and test samples
dti_training_drugs = dti_drugs[:int(len(dti_drugs) * ratio)]
dti_test_drugs = dti_drugs[int(len(dti_drugs) * ratio):]
print("len(dti_training_drugs):", len(dti_training_drugs))
print("len(dti_test_drugs):", len(dti_test_drugs))
                                 

In [ ]:
# filter out the drug screen by training and test with condition that meet both cell line test samples and drug test samples for test
final_training_drug_screen_df = dti_drug_overlap_df[dti_drug_overlap_df['depMapID'].isin(dti_training_samples) & dti_drug_overlap_df['BMGC_Drug_ID'].isin(dti_training_drugs)].reset_index(drop=True)
display(final_training_drug_screen_df)
# and calculate the final training drug ids and cell line ids
training_drug_ids = list(set(final_training_drug_screen_df['BMGC_Drug_ID']))
training_cell_line_ids = list(set(final_training_drug_screen_df['depMapID']))
print("len(training_drug_ids):", len(training_drug_ids))
print("len(training_cell_line_ids):", len(training_cell_line_ids))

In [ ]:
# the rest of the drug screen is the test drug screen
final_test_drug_screen_df = dti_drug_overlap_df[~dti_drug_overlap_df.index.isin(final_training_drug_screen_df.index)].reset_index(drop=True)
display(final_test_drug_screen_df)
# and calculate the final test drug ids and cell line ids
test_drug_ids = list(set(final_test_drug_screen_df['BMGC_Drug_ID']))
test_cell_line_ids = list(set(final_test_drug_screen_df['depMapID']))
print("len(test_drug_ids):", len(test_drug_ids))
print("len(test_cell_line_ids):", len(test_cell_line_ids))

### 8.3 Pretrain samples

In [ ]:
# get the rest of samples not using in dti task (difference between overlapped_omics_union_annotation_samples and dti_overlapped_samples_df)
rest_samples = list(set(overlapped_omics_union_annotation_samples) - set(dti_overlapped_samples))
print("len(rest_samples):", len(rest_samples))

In [ ]:
# get the pretraining samples by concatenating the training_cell_line_ids and rest_samples
pretraining_samples = list(set(training_cell_line_ids) | set(rest_samples))
print("len(pretraining_samples):", len(pretraining_samples))
print(pretraining_samples[:5])

In [ ]:
non_cancerous_samples_df = pd.read_csv('./raw_data/cell-lines-in-Non-Cancerous.csv')
# get the depmap id from the non_cancerous_samples_df
non_cancerous_samples = list(non_cancerous_samples_df['Depmap Id'])
# get the overlapped non_cancerous samples with the overlapped_omics_union_annotation_samples
overlapped_non_cancerous_samples = sorted(list(set(overlapped_omics_union_annotation_samples) & set(non_cancerous_samples)))
print("length of overlapped omics and annotation samples: ", len(overlapped_omics_union_annotation_samples))
print("length of overlapped non cancerous samples: ", len(overlapped_non_cancerous_samples))

In [ ]:
# based on the non_cancerous_samples, get the non_cancerous_pretrain_samples in the pretraining_samples
non_cancerous_pretrain_samples = sorted(list(set(pretraining_samples) & set(overlapped_non_cancerous_samples)))
print("length of non cancerous pretrain samples: ", len(non_cancerous_pretrain_samples))
cancerous_pretrain_samples = sorted(list(set(pretraining_samples) - set(non_cancerous_pretrain_samples)))
print("length of cancerous pretrain samples: ", len(cancerous_pretrain_samples))

### 8.4 Pretrain integration

In [ ]:
maped_methy_df = final_merged_methy_df.rename(columns=methy_map_dict)
display(maped_methy_df)
# insert the samples that is in the overlapped_samples but not in the maped_methy_df
for sample in pretraining_samples:
    if sample not in maped_methy_df.columns:
        maped_methy_df[sample] = 0.0
# filter out the samples that is not in the pretraining_samples
pretrain_methy_df = maped_methy_df[['BioMedGraphica_Conn_ID'] + pretraining_samples]
display(pretrain_methy_df)

In [ ]:
mapped_protein_df = final_merged_protein_df.rename(columns=protein_map_dict)
display(mapped_protein_df)
# insert the samples that is in the pretraining_samples but not in the mapped_protein_df
for sample in pretraining_samples:
    if sample not in mapped_protein_df.columns:
        mapped_protein_df[sample] = 0.0
# filter out the samples that is not in the pretraining_samples
pretrain_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + pretraining_samples]
display(pretrain_protein_df)

In [ ]:
# fill in the missing samples in the final_merged_gene_df for pretraining samples
for sample in pretraining_samples:
    if sample not in final_merged_gene_df.columns:
        final_merged_gene_df[sample] = 0.0
# fill in the missing samples in the final_merged_transcript_df for pretraining samples
for sample in pretraining_samples:
    if sample not in final_merged_transcript_df.columns:
        final_merged_transcript_df[sample] = 0.0
# get the final gene, transcript, drug dataframe by filtering the pretraining_samples
pretrain_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + pretraining_samples]
display(pretrain_gene_df)
pretrain_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + pretraining_samples]
display(pretrain_transcript_df)

In [ ]:
pretrain_omics_df = pd.concat([pretrain_methy_df, pretrain_gene_df, pretrain_transcript_df, pretrain_protein_df], axis=0).reset_index(drop=True)
display(pretrain_omics_df)

## 9. Convert to numpy files

In [ ]:
# create the numpy file folders with name DTI_data and Pretrain_data
import os
import shutil

# Create directories if they don't exist
os.makedirs('./DTI_data', exist_ok=True)
os.makedirs('./Pretrain_data', exist_ok=True)

### 9.1 Load BMGC

In [ ]:
bmgc_entity_df = pd.read_csv('./BioMedGraphica-Conn/Entity/BioMedGraphica_Conn_Entity.csv')
bmgc_relation_df = pd.read_csv('./BioMedGraphica-Conn/Relation/BioMedGraphica_Conn_Relation.csv')
display(bmgc_entity_df)
display(bmgc_relation_df)

In [ ]:
# check the null values in the bmgc_entity_df and bmgc_relation_df
print(bmgc_entity_df.isnull().sum())
print(bmgc_relation_df.isnull().sum())

In [ ]:
# print out the rows where BMGC_From_ID is null in the bmgc_relation_df
print(bmgc_relation_df[bmgc_relation_df['BMGC_From_ID'].isnull()])

### 9.2 DTI numpy files

#### 9.2.1 Merge files for X

In [ ]:
# keep the columns ['BioMedGraphica_Conn_ID'] in the bmgc_entity_df
dti_bmgc_entity_df = bmgc_entity_df[['BioMedGraphica_Conn_ID']].copy()
# left join the dti_bmgc_entity_df with the dti_omics_df on BioMedGraphica_Conn_ID
dti_bmgc_omics_df = pd.merge(dti_bmgc_entity_df, dti_omics_df, on='BioMedGraphica_Conn_ID', how='left')
# fill NaN values with 0.0
dti_bmgc_omics_df = dti_bmgc_omics_df.fillna(0.0)
display(dti_bmgc_omics_df)

#### 9.2.2 Add the index files for samples and nodes

In [ ]:
# create an index data file for cell lines and save it to the DTI_data folder
samples = dti_bmgc_omics_df.columns[1:]
# create sample index ranging from 0 to len(samples)-1
sample_index = np.arange(len(samples))
sample_index_data = pd.DataFrame({'Sample': samples, 'Index': sample_index})
display(sample_index_data)
sample_index_data.to_csv('./DTI_data/sample_index.csv', index=False)

# create an index data file for nodes and save it to the DTI_data folder
nodes = dti_bmgc_omics_df['BioMedGraphica_Conn_ID'].tolist()
# create nodes index ranging from 0 to len(nodes)-1
nodes_index = np.arange(len(nodes))
nodes_index_data = pd.DataFrame({'Node': nodes, 'Index': nodes_index})
nodes_index_data = pd.merge(nodes_index_data, bmgc_entity_df[['BioMedGraphica_Conn_ID', 'Type']], left_on='Node', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_Conn_ID'])
display(nodes_index_data)
nodes_index_data.to_csv('./DTI_data/nodes_index.csv', index=False)

#### 9.2.3 Convert to numpy files

In [ ]:
# convert the dti_bmgc_omics_df to numpy array aside from the first column (BioMedGraphica_Conn_ID) and save it to the DTI_data folder
dti_bmgc_omics_array = dti_bmgc_omics_df.iloc[:, 1:].to_numpy()
# transpose the array
dti_bmgc_omics_array_T = dti_bmgc_omics_array.T
print('The shape of dti_bmgc_omics_array_T is:', dti_bmgc_omics_array_T.shape)
print(dti_bmgc_omics_array_T)
# save the numpy array to the DTI_data folder
np.save('./DTI_data/dti_bmgc_omics.npy', dti_bmgc_omics_array_T)

In [ ]:
# save the dti_drug_overlap_df, final_training_drug_screen_df and final_test_drug_screen_df to the DTI_data folder
dti_drug_overlap_df.to_csv('./DTI_data/dti_drug_overlap.csv', index=False)
final_training_drug_screen_df.to_csv('./DTI_data/final_training_drug_screen.csv', index=False)
final_test_drug_screen_df.to_csv('./DTI_data/final_test_drug_screen.csv', index=False)
final_training_drug_screen_numpy_df = final_training_drug_screen_df[['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']].copy()
final_test_drug_screen_numpy_df = final_test_drug_screen_df[['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']].copy()
display(final_training_drug_screen_numpy_df)
display(final_test_drug_screen_numpy_df)

In [ ]:
# Create dictionaries using zip and dict()
sample_index_dict = dict(zip(sample_index_data['Sample'], sample_index_data['Index']))
# Print the first 10 items of sample_index_dict
print("First 10 items in sample_index_dict:")
print(list(sample_index_dict.items())[:10])

node_index_dict = dict(zip(nodes_index_data['Node'], nodes_index_data['Index']))
# Print the first 10 items of node_index_dict
print("\nFirst 10 items in node_index_dict:")
print(list(node_index_dict.items())[:10])

# Map the depMapID and [BMGC_Drug_ID, BMGC_Disease_ID] to the index
final_training_drug_screen_numpy_mapped_df = final_training_drug_screen_numpy_df.copy()
final_training_drug_screen_numpy_mapped_df['depMapID'] = final_training_drug_screen_numpy_mapped_df['depMapID'].map(sample_index_dict)
final_training_drug_screen_numpy_mapped_df['BMGC_Drug_ID'] = final_training_drug_screen_numpy_mapped_df['BMGC_Drug_ID'].map(node_index_dict)
final_training_drug_screen_numpy_mapped_df['BMGC_Disease_ID'] = final_training_drug_screen_numpy_mapped_df['BMGC_Disease_ID'].map(node_index_dict)

# check the null values in the final_training_drug_screen_numpy_mapped_df
print("Null values in final_training_drug_screen_numpy_mapped_df:")
print(final_training_drug_screen_numpy_mapped_df.isnull().sum())
display(final_training_drug_screen_numpy_mapped_df)

# convert the final_training_drug_screen_numpy_mapped_df to numpy array and save it to the DTI_data folder
final_training_drug_screen_numpy_mapped_array = final_training_drug_screen_numpy_mapped_df[['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']].to_numpy()
print('The shape of final_training_drug_screen_numpy_mapped_array is:', final_training_drug_screen_numpy_mapped_array.shape)
print(final_training_drug_screen_numpy_mapped_array)
# save the numpy array to the DTI_data folder
np.save('./DTI_data/final_training_drug_screen_numpy_mapped.npy', final_training_drug_screen_numpy_mapped_array)

In [ ]:
# Map the depMapID and [BMGC_Drug_ID, BMGC_Disease_ID] to the index for test set
final_test_drug_screen_numpy_mapped_df = final_test_drug_screen_numpy_df.copy()
final_test_drug_screen_numpy_mapped_df['depMapID'] = final_test_drug_screen_numpy_mapped_df['depMapID'].map(sample_index_dict)
final_test_drug_screen_numpy_mapped_df['BMGC_Drug_ID'] = final_test_drug_screen_numpy_mapped_df['BMGC_Drug_ID'].map(node_index_dict)
final_test_drug_screen_numpy_mapped_df['BMGC_Disease_ID'] = final_test_drug_screen_numpy_mapped_df['BMGC_Disease_ID'].map(node_index_dict)

# check the null values in the final_test_drug_screen_numpy_mapped_df
print("Null values in final_test_drug_screen_numpy_mapped_df:")
print(final_test_drug_screen_numpy_mapped_df.isnull().sum())
display(final_test_drug_screen_numpy_mapped_df)

# convert the final_test_drug_screen_numpy_mapped_df to numpy array and save it to the DTI_data folder
final_test_drug_screen_numpy_mapped_array = final_test_drug_screen_numpy_mapped_df[['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']].to_numpy()
print('The shape of final_test_drug_screen_numpy_mapped_array is:', final_test_drug_screen_numpy_mapped_array.shape)
print(final_test_drug_screen_numpy_mapped_array)
# save the numpy array to the DTI_data folder
np.save('./DTI_data/final_test_drug_screen_numpy_mapped.npy', final_test_drug_screen_numpy_mapped_array)

In [ ]:
# Convert the bmgc_relation_df to numpy array and save it to the DTI_data folder as the edge_index.npy
# keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
edge_index_df = bmgc_relation_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
edge_index_df['BMGC_From_ID'] = edge_index_df['BMGC_From_ID'].map(node_index_dict)
edge_index_df['BMGC_To_ID'] = edge_index_df['BMGC_To_ID'].map(node_index_dict)
# check the null values in the edge_index_df
print("Null values in edge_index_df:")
print(edge_index_df.isnull().sum())
display(edge_index_df)
# convert the edge_index_df to numpy array and save it to the DTI_data folder
edge_index_array = edge_index_df.to_numpy().T
print('The shape of edge_index_array is:', edge_index_array.shape)
print(edge_index_array)
# save the numpy array to the DTI_data folder
np.save('./DTI_data/edge_index.npy', edge_index_array)

In [ ]:
# generate the internal_edge_index by selecting bmgc_relation_df in Type ['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein']
internal_edge_index_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein'])].copy()
# keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
internal_edge_index_df = internal_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
internal_edge_index_df['BMGC_From_ID'] = internal_edge_index_df['BMGC_From_ID'].map(node_index_dict)
internal_edge_index_df['BMGC_To_ID'] = internal_edge_index_df['BMGC_To_ID'].map(node_index_dict)
# check the null values in the internal_edge_index_df
print("Null values in internal_edge_index_df:")
print(internal_edge_index_df.isnull().sum())
# convert the internal_edge_index_df to numpy array and save it to the DTI_data folder
internal_edge_index_array = internal_edge_index_df.to_numpy().T
print('The shape of internal_edge_index_array is:', internal_edge_index_array.shape)
print(internal_edge_index_array)
# save the numpy array to the DTI_data folder
np.save('./DTI_data/internal_edge_index.npy', internal_edge_index_array)

In [ ]:
# generate the ppi_edge_index by selecting bmgc_relation_df in Type ['Protein-Protein']
ppi_edge_index_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Protein-Protein'])].copy()
# keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
ppi_edge_index_df = ppi_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
ppi_edge_index_df['BMGC_From_ID'] = ppi_edge_index_df['BMGC_From_ID'].map(node_index_dict)
ppi_edge_index_df['BMGC_To_ID'] = ppi_edge_index_df['BMGC_To_ID'].map(node_index_dict)
# check the null values in the ppi_edge_index_df
print("Null values in ppi_edge_index_df:")
print(ppi_edge_index_df.isnull().sum())
# convert the ppi_edge_index_df to numpy array and save it to the DTI_data folder
ppi_edge_index_array = ppi_edge_index_df.to_numpy().T
print('The shape of ppi_edge_index_array is:', ppi_edge_index_array.shape)
print(ppi_edge_index_array)
# save the numpy array to the DTI_data folder
np.save('./DTI_data/ppi_edge_index.npy', ppi_edge_index_array)

#### 9.2.4 DTI data with name and desc

In [ ]:
# Create the name and desc files
bmgc_promoter_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_gene_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_transcript_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_protein_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_pathway_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Pathway/BioMedGraphica_Conn_Pathway_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_metabolite_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Metabolite/BioMedGraphica_Conn_Metabolite_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_microbiota_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Microbiota/BioMedGraphica_Conn_Microbiota_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_exposure_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Exposure/BioMedGraphica_Conn_Exposure_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_phenotype_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Phenotype/BioMedGraphica_Conn_Phenotype_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_disease_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Disease/BioMedGraphica_Conn_Disease_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_drug_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Drug/BioMedGraphica_Conn_Drug_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])

In [ ]:
# get the number of rows in each dataframe and sum the number of rows for those dataframes
print("bmgc_promoter_name_id_df:", len(bmgc_promoter_name_id_df))
print("bmgc_gene_name_id_df:", len(bmgc_gene_name_id_df))
print("bmgc_transcript_name_id_df:", len(bmgc_transcript_name_id_df))
print("bmgc_protein_name_id_df:", len(bmgc_protein_name_id_df))
print("bmgc_pathway_name_id_df:", len(bmgc_pathway_name_id_df))
print("bmgc_metabolite_name_id_df:", len(bmgc_metabolite_name_id_df))
print("bmgc_microbiota_name_id_df:", len(bmgc_microbiota_name_id_df))
print("bmgc_exposure_name_id_df:", len(bmgc_exposure_name_id_df))
print("bmgc_phenotype_name_id_df:", len(bmgc_phenotype_name_id_df))
print("bmgc_disease_name_id_df:", len(bmgc_disease_name_id_df))
print("bmgc_drug_name_id_df:", len(bmgc_drug_name_id_df))
# sum the number of rows for those dataframes
print("Total number of rows in all dataframes:", len(bmgc_promoter_name_id_df) + len(bmgc_gene_name_id_df) + len(bmgc_transcript_name_id_df) + len(bmgc_protein_name_id_df) + len(bmgc_pathway_name_id_df) + len(bmgc_metabolite_name_id_df) + len(bmgc_microbiota_name_id_df) + len(bmgc_exposure_name_id_df) + len(bmgc_phenotype_name_id_df) + len(bmgc_disease_name_id_df) + len(bmgc_drug_name_id_df))

In [ ]:
# left join those name_df into bmgc_entity_df
bmgc_entity_cp_df = bmgc_entity_df[['BioMedGraphica_Conn_ID']].copy()
display(bmgc_entity_cp_df)
# concatenate all the name_id_df into one dataframe
bmgc_name_id_tmp_df = pd.concat([bmgc_promoter_name_id_df, bmgc_gene_name_id_df, bmgc_transcript_name_id_df, bmgc_protein_name_id_df, bmgc_pathway_name_id_df, bmgc_metabolite_name_id_df, bmgc_microbiota_name_id_df, bmgc_exposure_name_id_df, bmgc_phenotype_name_id_df, bmgc_disease_name_id_df, bmgc_drug_name_id_df], ignore_index=True)
display(bmgc_name_id_tmp_df)
# left join the bmgc_name_id_tmp_df into bmgc_entity_cp_df on BioMedGraphica_Conn_ID
bmgc_entity_cp_df = pd.merge(bmgc_entity_cp_df, bmgc_name_id_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
display(bmgc_entity_cp_df)
bmgc_entity_cp_df.to_csv('./DTI_data/bmgc_name.csv', index=False)

In [ ]:
bmgc_promoter_desc_df = bmgc_promoter_name_id_df.drop(columns=['Names_and_IDs'], axis=1).copy()
bmgc_promoter_desc_df['Description'] = np.nan # add the Description column to bmgc_promoter_name_df with NaN values
bmgc_gene_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_transcript_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_protein_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_pathway_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Pathway/BioMedGraphica_Conn_Pathway_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_metabolite_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Metabolite/BioMedGraphica_Conn_Metabolite_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_microbiota_desc_df = bmgc_microbiota_name_id_df.drop(columns=['Names_and_IDs'], axis=1).copy()
bmgc_microbiota_desc_df['Description'] = np.nan # add the Description column to bmgc_microbiota_name_df with NaN values
bmgc_exposure_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Exposure/BioMedGraphica_Conn_Exposure_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_phenotype_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Phenotype/BioMedGraphica_Conn_Phenotype_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_disease_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Disease/BioMedGraphica_Conn_Disease_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_drug_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Drug/BioMedGraphica_Conn_Drug_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])

In [ ]:
bmgc_desc_tmp_df = pd.concat([bmgc_promoter_desc_df, bmgc_gene_desc_df, bmgc_transcript_desc_df, bmgc_protein_desc_df, bmgc_pathway_desc_df, bmgc_metabolite_desc_df, bmgc_microbiota_desc_df, bmgc_exposure_desc_df, bmgc_phenotype_desc_df, bmgc_disease_desc_df, bmgc_drug_desc_df], ignore_index=True)
display(bmgc_desc_tmp_df)
# left join the bmgc_desc_tmp_df into bmgc_entity_cp_df on BioMedGraphica_Conn_ID
bmgc_entity_cp_df = bmgc_entity_df[['BioMedGraphica_Conn_ID']].copy()
bmgc_desc_df = pd.merge(bmgc_entity_cp_df, bmgc_desc_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
# check the null values in the bmgc_desc_df
print("Null values in bmgc_desc_df:")
print(bmgc_desc_df.isnull().sum())
# fill the NaN values in the Description column with empty string
bmgc_desc_df['Description'] = bmgc_desc_df['Description'].fillna(' ')
# recheck the null values in the bmgc_desc_df
print("Null values in bmgc_desc_df:")
print(bmgc_desc_df.isnull().sum())
display(bmgc_desc_df)
bmgc_desc_df.to_csv('./DTI_data/bmgc_desc.csv', index=False)    

In [ ]:
# Create a meta data file in the folder DTI_data to explain the dti_bmgc_omics.npy, final_training_drug_screen_numpy_mapped.npy, final_test_drug_screen_numpy_mapped.npy for following:
# 1. dti_bmgc_omics.npy: each row is a sample, and each column is a feature. The row index corresponds to the sample index in the sample_index.csv file. The column index corresponds to the feature index in the nodes_index.csv file.
# 2. final_training_drug_screen_numpy_mapped.npy: each row is a drug-screening sample, and it has 4 columns: depMapID sample index, BMGC_Drug_ID drug index, BMGC_Disease_ID disease index, AUC value. 
# By using the sample index to fetch the sample in dti_bmgc_omics.npy, and using the drug index and disease index to mark the drug and disease in the dti_bmgc_omics.npy as 1, and the rest as 0.
# 3. final_test_drug_screen_numpy_mapped.npy: same as final_training_drug_screen_numpy_mapped.npy, but for test set.

### 9.3 Pretrain numpy files

#### 9.3.1 Pretraining data for omics

In [ ]:
# load entity type ['Promoter', 'Gene', 'Transcript', 'Protein'] for bmgc_entity_df
bmgc_entity_df = pd.read_csv('./BioMedGraphica-Conn/Entity/BioMedGraphica_Conn_Entity.csv')
# check if there is any null values in the bmgc_entity_df
print(bmgc_entity_df.isnull().sum())
# filter out the rows with Type in the list ['promoter', 'gene', 'transcript', 'protein']
bmgc_omics_df = bmgc_entity_df[bmgc_entity_df['Type'].isin(['Promoter', 'Gene', 'Transcript', 'Protein'])].reset_index(drop=True)
# check if there is any null values in the bmgc_omics_df
print(bmgc_omics_df.isnull().sum())
display(bmgc_omics_df)

In [ ]:
# load the relation
bmgc_relation_df = pd.read_csv('./BioMedGraphica-Conn/Relation/BioMedGraphica_Conn_Relation.csv')
# check if there is any null values in the bmgc_relation_df
print(bmgc_relation_df.isnull().sum())

In [ ]:
# filter our the rows with Relation type in the list ['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein', 'Protein-Protein']
bmgc_omics_relation_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein', 'Protein-Protein'])].reset_index(drop=True)
# check if there is any null values in the bmgc_omics_relation_df
print(bmgc_omics_relation_df.isnull().sum())
display(bmgc_omics_relation_df)

In [ ]:
pretrain_bmgc_omics_df = pd.merge(bmgc_omics_df['BioMedGraphica_Conn_ID'], pretrain_omics_df, on='BioMedGraphica_Conn_ID', how='left')
# sort the columns orders ACH-xxxxxxx by ascending order aside from the first column (BioMedGraphica_Conn_ID)
pretrain_bmgc_omics_df = pretrain_bmgc_omics_df[['BioMedGraphica_Conn_ID'] + sorted(pretrain_bmgc_omics_df.columns[1:])]
display(pretrain_bmgc_omics_df)
# check if there is any null values in the pretrain_bmgc_omics_df and null value over whole dataframe
print(pretrain_bmgc_omics_df.isnull().sum())
print(pretrain_bmgc_omics_df.isnull().sum().sum())

#### 9.3.2 Add the index files for samples and nodes

In [ ]:
# Add the index for cell lines and entities in the pretrain_bmgc_omics_df
samples = pretrain_bmgc_omics_df.columns[1:]
# create sample index ranging from 0 to len(samples)-1
sample_index = np.arange(len(samples))
sample_index_data = pd.DataFrame({'Sample': samples, 'Index': sample_index})
display(sample_index_data)
# save the sample index data to the Pretrain_data folder
sample_index_data.to_csv('./Pretrain_data/sample_index.csv', index=False)

# Create the index data for nodes and save it to the Pretrain_data folder
omics_nodes = pretrain_bmgc_omics_df['BioMedGraphica_Conn_ID'].tolist()
# create nodes index ranging from 0 to len(nodes)-1
omics_nodes_index = np.arange(len(omics_nodes))
omics_nodes_index_data = pd.DataFrame({'Node': omics_nodes, 'Index': omics_nodes_index})
omics_nodes_index_data = pd.merge(omics_nodes_index_data, bmgc_omics_df[['BioMedGraphica_Conn_ID', 'Type']], left_on='Node', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_Conn_ID'])
display(omics_nodes_index_data)
# save the nodes index data to the Pretrain_data folder
omics_nodes_index_data.to_csv('./Pretrain_data/omics_nodes_index.csv', index=False)

#### 9.3.3 Convert to numpy files

In [ ]:
# convert the pretrain_bmgc_omics_df to numpy array aside from the first column (BioMedGraphica_Conn_ID) and save it to the Pretrain_data folder
pretrain_bmgc_omics_array = pretrain_bmgc_omics_df.iloc[:, 1:].to_numpy()
# transpose the array
pretrain_bmgc_omics_array_T = pretrain_bmgc_omics_array.T
print('The shape of pretrain_bmgc_omics_array_T is:', pretrain_bmgc_omics_array_T.shape)
print(pretrain_bmgc_omics_array_T)
# save the numpy array to the Pretrain_data folder
np.save('./Pretrain_data/pretrain_bmgc_omics.npy', pretrain_bmgc_omics_array_T)

In [ ]:
# non-cancerous cell lines
non_cancerous_samples_df = pd.read_csv('./raw_data/cell-lines-in-Non-Cancerous.csv')
# Add the column 'Status' to the non_cancerous_samples_df with value of 0
non_cancerous_samples_df['Status'] = 0
# keep the columns ['Depmap Id', 'Status']
non_cancerous_samples_df = non_cancerous_samples_df[['Depmap Id', 'Status']].copy()
# display(non_cancerous_samples_df)

# join the non_cancerous_samples_df with sample_index_data on Depmap Id
pretrain_label_df = pd.merge(sample_index_data, non_cancerous_samples_df, left_on='Sample', right_on='Depmap Id', how='left')
# display(pretrain_label_df)
# fill NaN values with 1 in Status column
pretrain_label_df['Status'] = pretrain_label_df['Status'].fillna(1)
# keep the columns['Sample', 'Index', 'Status']
pretrain_label_df = pretrain_label_df[['Sample', 'Index', 'Status']].copy()
display(pretrain_label_df)

# count the number of 0s and 1s in the Status column
print(pretrain_label_df['Status'].value_counts())

##### 9.3.3.1 Upsampling training

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Step 1: Split data into train and test sets while preserving class distribution
train_df, test_df = train_test_split(
    pretrain_label_df,
    test_size=0.2,  # 80% train, 20% test
    random_state=42,
    stratify=pretrain_label_df['Status']  # Ensures same class distribution in train/test
)

# Reset indices
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Display original distributions
print("Before upsampling:")
print("Training set class distribution:")
print(train_df['Status'].value_counts())
print(train_df['Status'].value_counts(normalize=True))

# Step 2: Separate cancerous and non-cancerous samples in the training set
train_cancerous = train_df[train_df['Status'] == True]
train_non_cancerous = train_df[train_df['Status'] == False]

# Count samples
n_train_cancerous = len(train_cancerous)
n_train_non_cancerous = len(train_non_cancerous)

# Step 3: Upsample the minority class in the training set only
if n_train_non_cancerous < n_train_cancerous:
    # Upsample non-cancerous samples
    train_non_cancerous_upsampled = resample(
        train_non_cancerous,
        replace=True,
        n_samples=n_train_cancerous,
        random_state=42
    )
    # Combine upsampled non-cancerous with original cancerous
    balanced_train_df = pd.concat([train_cancerous, train_non_cancerous_upsampled])
else:
    # Upsample cancerous samples
    train_cancerous_upsampled = resample(
        train_cancerous,
        replace=True, 
        n_samples=n_train_non_cancerous,
        random_state=42
    )
    # Combine upsampled cancerous with original non-cancerous
    balanced_train_df = pd.concat([train_cancerous_upsampled, train_non_cancerous])

# Shuffle the balanced training set
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display final distributions
print("\nAfter upsampling:")
print("Training set class distribution:")
print(balanced_train_df['Status'].value_counts())
print(balanced_train_df['Status'].value_counts(normalize=True))
print("\nTest set class distribution (unchanged):")
print(test_df['Status'].value_counts())
print(test_df['Status'].value_counts(normalize=True))

# Optionally save the datasets
balanced_train_df.to_csv('./Pretrain_data/balanced_train.csv', index=False)
test_df.to_csv('./Pretrain_data/balanced_test.csv', index=False)

In [ ]:
# Function to check for overlapping samples between two dataframes
def check_overlapping_samples(df1, df2, label='Status'):
    """
    Check for overlapping samples between two dataframes
    
    Args:
        df1, df2: The dataframes to compare
        label: Optional column to exclude from comparison
    
    Returns:
        Number of overlapping samples and the overlapping rows
    """
    # Get columns to compare (all except possibly the label)
    cols_to_compare = [col for col in df1.columns if col != label]
    
    # Method 1: Direct merge approach
    merged = pd.merge(df1[cols_to_compare], df2[cols_to_compare], 
                      how='inner', indicator=False)
    
    # If we have any rows in the merged DataFrame, those are duplicates
    num_duplicates = len(merged)
    
    # Method 2: Convert to tuples and use set intersection
    # This is a verification step - should match the first method
    df1_tuples = set(map(tuple, df1[cols_to_compare].values))
    df2_tuples = set(map(tuple, df2[cols_to_compare].values))
    intersection = df1_tuples.intersection(df2_tuples)
    
    print(f"Number of duplicated samples: {num_duplicates}")
    print(f"Verification: Set intersection found {len(intersection)} duplicates")
    
    # Return the duplicated rows for inspection if any exist
    if num_duplicates > 0:
        return merged.drop_duplicates()
    return pd.DataFrame()

# Check for overlapping samples between our balanced training set and test set
overlapping_samples = check_overlapping_samples(balanced_train_df, test_df)

if len(overlapping_samples) > 0:
    print("WARNING: Found overlapping samples between training and test sets!")
    print("Sample of duplicated data points:")
    print(overlapping_samples.head())
else:
    print("No overlapping samples found between training and test sets. Data split is clean.")
    
# Additional check: Double-check that upsampling hasn't created duplicates within training set
train_duplicates = balanced_train_df.duplicated().sum()
print(f"\nNumber of duplicated rows within training set: {train_duplicates}")

# This is expected since we've upsampled and should see duplicates in training
if train_duplicates > 0:
    print("Note: Duplicates in training set are expected due to upsampling.")

In [ ]:
import numpy as np

# Extract columns from balanced training set and convert to numpy array
balanced_train_numpy_df = balanced_train_df[['Index', 'Status']].copy()
balanced_train_numpy = balanced_train_numpy_df.to_numpy()
print('The shape of balanced_train_numpy is:', balanced_train_numpy.shape)
print(balanced_train_numpy[:10])  # Print first 10 rows as a sample

# Save the balanced training numpy array
np.save('./Pretrain_data/balanced_train.npy', balanced_train_numpy)

# Extract columns from test set and convert to numpy array 
test_numpy_df = test_df[['Index', 'Status']].copy()
test_numpy = test_numpy_df.to_numpy()
print('\nThe shape of test_numpy is:', test_numpy.shape)
print(test_numpy[:10])  # Print first 10 rows as a sample

# Save the test numpy array
np.save('./Pretrain_data/balanced_test.npy', test_numpy)

print("\nNumPy arrays successfully saved to Pretrain_data folder.")

##### 9.3.3.2 Upsampliing the train minority and Downsampling test majority

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Step 1: Split data into train and test sets while preserving class distribution
train_df, test_df = train_test_split(
    pretrain_label_df,
    test_size=0.2,  # 80% train, 20% test
    random_state=42,
    stratify=pretrain_label_df['Status']  # Ensures same class distribution in train/test
)

# Reset indices
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Display original distributions
print("Before upsampling/downsampling:")
print("Training set class distribution:")
print(train_df['Status'].value_counts())
print(train_df['Status'].value_counts(normalize=True))
print("\nTest set original class distribution:")
print(test_df['Status'].value_counts())
print(test_df['Status'].value_counts(normalize=True))

# Step 2: Process training set - Upsample minority class to 30% (70/30 ratio)
train_cancerous = train_df[train_df['Status'] == True]
train_non_cancerous = train_df[train_df['Status'] == False]

# Count samples
n_train_cancerous = len(train_cancerous)
n_train_non_cancerous = len(train_non_cancerous)

# Calculate target numbers for 70-30 ratio in training set
if n_train_cancerous > n_train_non_cancerous:
    # Cancerous is majority (70%), non-cancerous is minority (30%)
    target_non_cancerous = int(n_train_cancerous * 3/7)  # To achieve 70-30 ratio
    
    # Upsample non-cancerous samples
    train_non_cancerous_upsampled = resample(
        train_non_cancerous,
        replace=True,
        n_samples=target_non_cancerous,
        random_state=42
    )
    # Combine upsampled non-cancerous with original cancerous
    balanced_train_df = pd.concat([train_cancerous, train_non_cancerous_upsampled])
else:
    # Non-cancerous is majority (70%), cancerous is minority (30%)
    target_cancerous = int(n_train_non_cancerous * 3/7)  # To achieve 70-30 ratio
    
    # Upsample cancerous samples
    train_cancerous_upsampled = resample(
        train_cancerous,
        replace=True, 
        n_samples=target_cancerous,
        random_state=42
    )
    # Combine upsampled cancerous with original non-cancerous
    balanced_train_df = pd.concat([train_cancerous_upsampled, train_non_cancerous])

# Step 3: Process test set - Downsample majority class to achieve 70/30 ratio
test_cancerous = test_df[test_df['Status'] == True]
test_non_cancerous = test_df[test_df['Status'] == False]

n_test_cancerous = len(test_cancerous)
n_test_non_cancerous = len(test_non_cancerous)

if n_test_cancerous > n_test_non_cancerous:
    # Cancerous is majority - downsample to 70%
    # Calculate how many cancerous samples we need for a 70/30 ratio
    target_cancerous = int(n_test_non_cancerous * 7/3)
    
    # Downsample cancerous samples without replacement
    test_cancerous_downsampled = test_cancerous.sample(
        n=target_cancerous,
        random_state=42
    )
    # Combine downsampled cancerous with original non-cancerous
    balanced_test_df = pd.concat([test_cancerous_downsampled, test_non_cancerous])
else:
    # Non-cancerous is majority - downsample to 70%
    # Calculate how many non-cancerous samples we need for a 70/30 ratio
    target_non_cancerous = int(n_test_cancerous * 7/3)
    
    # Downsample non-cancerous samples without replacement
    test_non_cancerous_downsampled = test_non_cancerous.sample(
        n=target_non_cancerous,
        random_state=42
    )
    # Combine downsampled non-cancerous with original cancerous
    balanced_test_df = pd.concat([test_cancerous, test_non_cancerous_downsampled])

# Shuffle both datasets
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_test_df = balanced_test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display final distributions
print("\nAfter processing:")
print("Training set class distribution:")
print(balanced_train_df['Status'].value_counts())
print(balanced_train_df['Status'].value_counts(normalize=True))
print("\nTest set class distribution:")
print(balanced_test_df['Status'].value_counts())
print(balanced_test_df['Status'].value_counts(normalize=True))

# Save the datasets
balanced_train_df.to_csv('./Pretrain_data/balanced_train.csv', index=False)
balanced_test_df.to_csv('./Pretrain_data/balanced_test.csv', index=False)

In [ ]:
import numpy as np

# Extract columns from balanced training set and convert to numpy array
balanced_train_numpy_df = balanced_train_df[['Index', 'Status']].copy()
balanced_train_numpy = balanced_train_numpy_df.to_numpy()
print('The shape of balanced_train_numpy is:', balanced_train_numpy.shape)
print('Class distribution in balanced training set:')
print(np.bincount(balanced_train_numpy[:, 1].astype(int)))
print('Sample rows:')
print(balanced_train_numpy[:10])  # Print first 10 rows as a sample

# Save the balanced training numpy array
np.save('./Pretrain_data/balanced_train.npy', balanced_train_numpy)

# Extract columns from balanced test set and convert to numpy array 
balanced_test_numpy_df = balanced_test_df[['Index', 'Status']].copy()
balanced_test_numpy = balanced_test_numpy_df.to_numpy()
print('\nThe shape of balanced_test_numpy is:', balanced_test_numpy.shape)
print('Class distribution in balanced test set:')
print(np.bincount(balanced_test_numpy[:, 1].astype(int)))
print('Sample rows:')
print(balanced_test_numpy[:10])  # Print first 10 rows as a sample

# Save the balanced test numpy array
np.save('./Pretrain_data/balanced_test.npy', balanced_test_numpy)

print("\nNumPy arrays successfully saved to Pretrain_data folder with 70/30 class distribution.")

# Optional: Verify the saved files
train_loaded = np.load('./Pretrain_data/balanced_train.npy')
test_loaded = np.load('./Pretrain_data/balanced_test.npy')

print("\nVerification of saved files:")
print(f"Train set shape: {train_loaded.shape}")
print(f"Test set shape: {test_loaded.shape}")
print("Class distributions after loading:")
print(f"Train: {np.bincount(train_loaded[:, 1].astype(int))}")
print(f"Test: {np.bincount(test_loaded[:, 1].astype(int))}")

In [ ]:
# Create dictionaries using zip and dict()
sample_index_dict = dict(zip(sample_index_data['Sample'], sample_index_data['Index']))
# Print the first 10 items of sample_index_dict
print("First 10 items in sample_index_dict:")
print(list(sample_index_dict.items())[:10])
print(len(sample_index_dict))

node_index_dict = dict(zip(nodes_index_data['Node'], nodes_index_data['Index']))
# Print the first 10 items of node_index_dict
print("\nFirst 10 items in node_index_dict:")
print(list(node_index_dict.items())[:10])
print(len(node_index_dict))

In [ ]:
# Convert the bmgc_omics_relation_df to numpy array and save it to the Pretrain_data folder as the edge_index.npy
# Keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
edge_index_df = bmgc_omics_relation_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()

# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
edge_index_df['BMGC_From_ID'] = edge_index_df['BMGC_From_ID'].map(node_index_dict) # Can use the same node_index_dict with dti_samples
edge_index_df['BMGC_To_ID'] = edge_index_df['BMGC_To_ID'].map(node_index_dict)

# check the null values in the edge_index_df
print("Null values in edge_index_df:")
print(edge_index_df.isnull().sum())
# print the rows with null values in column BMGC_From_ID the edge_index_df
print(edge_index_df[edge_index_df['BMGC_From_ID'].isnull()])
display(edge_index_df)
# convert the edge_index_df to numpy array and save it to the Pretrain_data folder
edge_index_array = edge_index_df.to_numpy().T
print('The shape of edge_index_array is:', edge_index_array.shape)
print(edge_index_array[:10])  # Print first 10 rows as a sample
# save the numpy array to the Pretrain_data folder
np.save('./Pretrain_data/edge_index.npy', edge_index_array)

In [ ]:
# filter out the internal_edge_index by Type belongs to ['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein']
internal_edge_index_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'].isin(['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein'])].reset_index(drop=True)
internal_edge_index_df = internal_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
internal_edge_index_df['BMGC_From_ID'] = internal_edge_index_df['BMGC_From_ID'].map(node_index_dict)
internal_edge_index_df['BMGC_To_ID'] = internal_edge_index_df['BMGC_To_ID'].map(node_index_dict)
display(internal_edge_index_df)
# convert the internal_edge_index_df to numpy array and save it to the Pretrain_data folder
internal_edge_index_array = internal_edge_index_df.to_numpy().T
print('The shape of internal_edge_index_array is:', internal_edge_index_array.shape)
print(internal_edge_index_array[:10])  # Print first 10 rows as a sample
# save the numpy array to the Pretrain_data folder
np.save('./Pretrain_data/internal_edge_index.npy', internal_edge_index_array)

In [ ]:
# filter out the ppi_edge_index by Type belongs to ['Protein-Protein']
ppi_edge_index_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'].isin(['Protein-Protein'])].reset_index(drop=True)
ppi_edge_index_df = ppi_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
ppi_edge_index_df['BMGC_From_ID'] = ppi_edge_index_df['BMGC_From_ID'].map(node_index_dict)
ppi_edge_index_df['BMGC_To_ID'] = ppi_edge_index_df['BMGC_To_ID'].map(node_index_dict)
display(ppi_edge_index_df)
# convert the ppi_edge_index_df to numpy array and save it to the Pretrain_data folder
ppi_edge_index_array = ppi_edge_index_df.to_numpy().T
print('The shape of ppi_edge_index_array is:', ppi_edge_index_array.shape)
print(ppi_edge_index_array[:10])  # Print first 10 rows as a sample
# save the numpy array to the Pretrain_data folder
np.save('./Pretrain_data/ppi_edge_index.npy', ppi_edge_index_array)

#### 9.3.4 Pretrain data with name and desc

In [ ]:
bmgc_omics_df

In [ ]:
# Create the name and desc files
bmgc_promoter_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_gene_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_transcript_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_protein_name_id_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])

bmgc_omics_name_id_tmp_df = pd.concat([bmgc_promoter_name_id_df, bmgc_gene_name_id_df, bmgc_transcript_name_id_df, bmgc_protein_name_id_df], axis=0).reset_index(drop=True)
bmgc_omics_name_id_df = pd.merge(bmgc_omics_df[['BioMedGraphica_Conn_ID']], bmgc_omics_name_id_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
# check the null values in the bmgc_omics_name_id_df
print("Null values in bmgc_omics_name_id_df:")
print(bmgc_omics_name_id_df.isnull().sum())
# fill the NaN values in the Names_and_IDs column with empty string
bmgc_omics_name_id_df['Names_and_IDs'] = bmgc_omics_name_id_df['Names_and_IDs'].fillna(' ')
# recheck the null values in the bmgc_omics_name_id_df
print("Null values in bmgc_omics_name_id_df:")
print(bmgc_omics_name_id_df.isnull().sum())
display(bmgc_omics_name_id_df)
bmgc_omics_name_id_df.to_csv('./Pretrain_data/bmgc_omics_name.csv', index=False)

In [ ]:
bmgc_promoter_desc_df = bmgc_promoter_name_id_df.drop(columns=['Names_and_IDs'], axis=1).copy()
bmgc_promoter_desc_df['Description'] = np.nan # add the Description column to bmgc_promoter_name_df with NaN values
bmgc_gene_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'], axis=1)
bmgc_transcript_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'], axis=1)
bmgc_protein_desc_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'], axis=1)

# concat and drop BioMedGraphica_ID column
bmgc_omics_desc_tmp_df = pd.concat([bmgc_promoter_desc_df, bmgc_gene_desc_df, bmgc_transcript_desc_df, bmgc_protein_desc_df], axis=0).reset_index(drop=True)
# check the null values in the bmgc_omics_desc_tmp_df
print(bmgc_omics_desc_tmp_df.isnull().sum())
bmgc_omics_desc_df = pd.merge(bmgc_omics_df[['BioMedGraphica_Conn_ID']], bmgc_omics_desc_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
# check the null values in the bmgc_omics_desc_df
print(bmgc_omics_desc_df.isnull().sum())
# fill NaN values with empty string in Description column
bmgc_omics_desc_df['Description'] = bmgc_omics_desc_df['Description'].fillna(' ')
# recheck the null values in the bmgc_omics_desc_df
print(bmgc_omics_desc_df.isnull().sum())
display(bmgc_omics_desc_df)

bmgc_omics_desc_df.to_csv('./Pretrain_data/bmgc_omics_desc.csv', index=False)

## 10. Add the pretraining biomarkers

### 10.0 Entity markers

In [ ]:
# translation chain converging to the same node
# fetch the promoter, gene, transcript and protein entity alone
promoter_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Promoter'].copy()
gene_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Gene'].copy()
transcript_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Transcript'].copy()
protein_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Protein'].copy()

display(bmgc_omics_relation_df)
# recheck the null values in bmgc_omics_relation_df
print("Null values in bmgc_omics_relation_df:")
print(bmgc_omics_relation_df.isnull().sum())

# fetch the Promoter-Gene, Gene-Transcript, Transcript-Protein relation alone
promoter_gene_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Promoter-Gene'].copy()
gene_transcript_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Gene-Transcript'].copy()
transcript_protein_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Transcript-Protein'].copy()

In [ ]:
gene_transcript_entity_df = pd.merge(gene_entity_df, gene_transcript_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BioMedGraphica_Conn_ID', right_on='BMGC_From_ID', how='outer')
gene_transcript_protein_entity_df = pd.merge(gene_transcript_entity_df, transcript_protein_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BMGC_To_ID', right_on='BMGC_From_ID', how='outer')
# drop NaN values in BMGC_From_ID_x	BMGC_To_ID_x BMGC_From_ID_y	BMGC_To_ID_y
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.dropna(subset=['BMGC_From_ID_x', 'BMGC_To_ID_x', 'BMGC_From_ID_y', 'BMGC_To_ID_y']).reset_index(drop=True)
# keep the columns ['BioMedGraphica_Conn_ID', 'BMGC_To_ID_x', 'BMGC_To_ID_y'] and rename the columns to ['BMGC_GN_ID', 'BMGC_TS_ID', 'BMGC_PT_ID']
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df[['BioMedGraphica_Conn_ID', 'BMGC_To_ID_y']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_GN_ID', 'BMGC_To_ID_y': 'BMGC_PT_ID'}).sort_values(by='BMGC_GN_ID').reset_index(drop=True)
# drop duplicates rows in gene_transcript_protein_entity_df
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.drop_duplicates().reset_index(drop=True)
display(gene_transcript_protein_entity_df)

In [ ]:
# just copy gene_transcript_protein_entity_df as promoter_gene_transcript_protein_entity_df
bmgc_promoter_df = pd.read_csv('../BMG/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
bmgc_gene_df = pd.read_csv('../BMG/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
promoter_gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.copy()
promoter_gene_df = pd.concat([bmgc_promoter_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_PM_ID'}), bmgc_gene_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_GN_ID'})], axis=1)
promoter_protein_entity_df = pd.merge(promoter_gene_transcript_protein_entity_df, promoter_gene_df, left_on='BMGC_GN_ID', right_on='BMGC_GN_ID', how='left').drop(columns=['BMGC_GN_ID'])
promoter_protein_entity_df = promoter_protein_entity_df[['BMGC_PM_ID', 'BMGC_PT_ID']]
display(promoter_protein_entity_df)

In [ ]:
transcript_protein_entity_df = pd.merge(transcript_entity_df, transcript_protein_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BioMedGraphica_Conn_ID', right_on='BMGC_From_ID', how='outer')
# drop NaN rows in the BMGC_From_ID and	BMGC_To_ID
transcript_protein_entity_df = transcript_protein_entity_df.dropna(subset=['BMGC_From_ID', 'BMGC_To_ID']).reset_index(drop=True)
# keep the columns ['BioMedGraphica_Conn_ID', 'BMGC_To_ID'] and rename the columns to ['BMGC_TS_ID', 'BMGC_PT_ID']
transcript_protein_entity_df = transcript_protein_entity_df[['BioMedGraphica_Conn_ID', 'BMGC_To_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_TS_ID', 'BMGC_To_ID': 'BMGC_PT_ID'}).sort_values(by='BMGC_TS_ID').reset_index(drop=True)
# drop duplicates rows in transcript_protein_entity_df
transcript_protein_entity_df = transcript_protein_entity_df.drop_duplicates().reset_index(drop=True)
display(transcript_protein_entity_df)

In [ ]:
# keep the columns ['BioMedGraphica_Conn_ID'] and rename the columns to ['BMGC_PT_ID']
only_protein_entity_df = protein_entity_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_PT_ID'}).sort_values(by='BMGC_PT_ID').reset_index(drop=True)
display(only_protein_entity_df)

### 10.1 CRISPR top 100 gene entities

In [ ]:
import pandas as pd
raw_crispr_df = pd.read_csv('./raw_data/CRISPRGeneEffect.csv')
display(raw_crispr_df)

In [ ]:
# get the column names of the raw_crispr_df aside from the first column and convert this to a list
raw_crispr_df_columns = raw_crispr_df.columns[1:].tolist()
print(raw_crispr_df_columns)

In [ ]:
# First, set the "Unnamed: 0" column as the index:
raw_crispr_t_df = raw_crispr_df.set_index("Unnamed: 0")
# Then transpose the DataFrame:
raw_crispr_t_df = raw_crispr_t_df.transpose()
# move the index to a column and rename it as the gene_name
raw_crispr_t_df.reset_index(inplace=True)
raw_crispr_t_df.rename(columns={'index': 'gene_name'}, inplace=True)
# column gene_name should be kept with the content by removing "()" and remove and " " content
raw_crispr_t_df['gene_name'] = raw_crispr_t_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Optionally, if you want to view the result:
display(raw_crispr_t_df)

In [ ]:
# Identify columns to convert (all except 'gene_name')
cols_to_convert = raw_crispr_t_df.columns.difference(['gene_name'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_crispr_t_df[cols_to_convert] = raw_crispr_t_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_crispr_t_df = raw_crispr_t_df.fillna(0.0)
display(raw_crispr_t_df)

In [ ]:
# groupby the gene_name and calculate the mean
crispr_df = raw_crispr_t_df.groupby('gene_name', as_index=False).mean()
display(crispr_df)

In [ ]:
bmg_gene_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_gene_df = bmg_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_gene_df)

In [ ]:
# merge the biomedgraphica_id with the crispr_df
merged_crispr_df = pd.merge(bmg_gene_df, crispr_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merged_crispr_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merged_crispr_df)

In [ ]:
# map the gene entity with gene_transcript_protein_entity_df
protein_crispr_df = pd.merge(gene_transcript_protein_entity_df, merged_crispr_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_Conn_ID'])
# and drop nan in any rows from protein_crispr_df
protein_crispr_df = protein_crispr_df.dropna().reset_index(drop=True)
display(protein_crispr_df)

In [ ]:
overlapped_omics_union_annotation_samples

In [ ]:
# Keep BioMedGraphica_Conn_ID and columns that are in overlapped_omics_union_annotation_samples
# For CRISPR data
columns_to_keep = ['BMGC_GN_ID', 'BMGC_PT_ID'] + [col for col in protein_crispr_df.columns if col in overlapped_omics_union_annotation_samples]
processed_crispr_df = protein_crispr_df[columns_to_keep]
display(processed_crispr_df)
processed_crispr_df.to_csv('./Pretrain_data/processed_crispr.csv', index=False)

In [ ]:
# Get the index of the cell line id 'ACH-000001'
cell_line_ach_id = 'ACH-000001'
cell_line_index = sample_index_data[sample_index_data['Sample'] == cell_line_ach_id]['Index'].values[0]

# Get the values of the cell line id 'ACH-000001' in the processed_crispr_df
# Sort by ascending values and keep only the first 100 rows (lowest values)
top_pt = 200
sample_pt_crispr_df = processed_crispr_df[['BMGC_PT_ID', cell_line_ach_id]].sort_values(by=cell_line_ach_id, ascending=True).head(top_pt).reset_index(drop=True)

# display the sample_pt_crispr_df
display(sample_pt_crispr_df)

In [ ]:
pt_list = sample_pt_crispr_df['BMGC_PT_ID'].tolist()
t_list = ['BMGC_PT013573', 'BMGC_PT013845', 'BMGC_PT059302', 'BMGC_PT086949', 'BMGC_PT087523', 'BMGC_PT019258', 'BMGC_PT015912', 'BMGC_PT032626', 'BMGC_PT078942', 'BMGC_PT022195', 'BMGC_PT043295', 'BMGC_PT092576', 'BMGC_PT040217', 'BMGC_PT016891', 'BMGC_PT017110', 'BMGC_PT042786', 'BMGC_PT017001', 'BMGC_PT015387', 'BMGC_PT018843', 'BMGC_PT015056', 'BMGC_PT014699', 'BMGC_PT015788', 'BMGC_PT049851', 'BMGC_PT019516', 'BMGC_PT017890', 'BMGC_PT014340', 'BMGC_PT035183', 'BMGC_PT034914', 'BMGC_PT018061', 'BMGC_PT103918', 'BMGC_PT014286', 'BMGC_PT032427', 'BMGC_PT013870', 'BMGC_PT013144', 'BMGC_PT018905', 'BMGC_PT036177', 'BMGC_PT034571', 'BMGC_PT011892', 'BMGC_PT010478', 'BMGC_PT046227', 'BMGC_PT031193', 'BMGC_PT017267', 'BMGC_PT017158', 'BMGC_PT095399', 'BMGC_PT015369', 'BMGC_PT049534', 'BMGC_PT048497', 'BMGC_PT073677', 'BMGC_PT036916', 'BMGC_PT027174']
# check the overlapped pt between pt_list and t_list
overlapped_pt = set(pt_list).intersection(set(t_list))
print("Overlapped pt between pt_list and t_list:", overlapped_pt)

In [ ]:
# gpt_top100_df = pd.read_csv('../ACH000001_top100_proteins.txt', sep='\t', header=None)
# # give the name to the none header columns
# gpt_top100_df.columns = ['Gene_Symbol']
# # remove suffix before '. ' in the column content
# gpt_top100_df['Gene_Symbol'] = gpt_top100_df['Gene_Symbol'].apply(lambda x: x.split('. ')[1])
# # display(gpt_top100_df)
# # Join the gene_symbol with bmgc_protein_entity
# bmgc_protein_tmp_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')
# bmgc_protein_tmp_df = bmgc_protein_tmp_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
# # Get the inner join with the bmgc_protein_tmp_df and gpt_top100_df
# gpt_top100_df = pd.merge(bmgc_protein_tmp_df, gpt_top100_df, left_on='HGNC_Symbol', right_on='Gene_Symbol', how='inner')
# # display(gpt_top100_df)
# gpt_pt_list = gpt_top100_df['BioMedGraphica_Conn_ID'].tolist()

# # check the overlapped pt between pt_list and gpt_pt_list
# overlapped_gpt_pt = set(pt_list).intersection(set(gpt_pt_list))
# print("Overlapped pt between pt_list and gpt_pt_list:", overlapped_gpt_pt)

### 10.2 RNAi top 100 transcript entities

In [ ]:
raw_rna_combined_df = pd.read_csv('./raw_data/D2_combined_gene_dep_scores.csv')
# fill NaN values with 0.0
raw_rna_combined_df = raw_rna_combined_df.fillna(0.0)
display(raw_rna_combined_df)

In [ ]:
# Move the Unnamed: 0 to a column and rename it as the gene_name
raw_rna_combined_df.rename(columns={'Unnamed: 0': 'gene_name'}, inplace=True)
# Clean the gene_name column by removing parentheses and extra spaces
raw_rna_combined_df['gene_name'] = raw_rna_combined_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Display the result
display(raw_rna_combined_df)

In [ ]:
# get the overlapped cell lines in methy_samples with cell_line_anno_df by merge on cell line id
rna_combined_samples = raw_rna_combined_df.columns[1:].tolist()
# first format the methy_samples as dataframe
rna_combined_samples_df = pd.DataFrame(rna_combined_samples, columns=['CCLE_Name'])
# merge the methy_samples_df with the cell_line_anno_df
rna_combined_samples_anno_df = pd.merge(rna_combined_samples_df, cell_line_anno_df, left_on='CCLE_Name', right_on='CCLE_ID', how='inner')
display(rna_combined_samples_anno_df)

# get the map dictionary from the ccle id to depmap id
rna_combined_samples_map_dict = dict(zip(rna_combined_samples_anno_df['CCLE_Name'], rna_combined_samples_anno_df['depMapID']))
print(rna_combined_samples_map_dict)

In [ ]:
maped_rna_combined_df = raw_rna_combined_df.rename(columns=rna_combined_samples_map_dict)
columns_to_keep = ['gene_name'] + sorted([col for col in maped_rna_combined_df.columns if col in overlapped_omics_union_annotation_samples])
filtered_rna_df = maped_rna_combined_df[columns_to_keep]
display(filtered_rna_df)

In [ ]:
bmg_transcript_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_transcript_df = bmg_transcript_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_transcript_df)

In [ ]:
# merge the biomedgraphica_id with the raw_transcript_df
merge_rna_df = pd.merge(bmg_transcript_df, filtered_rna_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merge_rna_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merge_rna_df)

In [ ]:
# map the transcript entity with transcript_protein_entity_df
protein_rna_df = pd.merge(transcript_protein_entity_df, merge_rna_df, left_on='BMGC_TS_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_Conn_ID'])
# and drop nan in any rows from protein_rna_df
processed_rna_df = protein_rna_df.dropna().reset_index(drop=True)
display(processed_rna_df)
processed_rna_df.to_csv('./Pretrain_data/processed_rna.csv', index=False)

In [ ]:
processed_crispr_samples = processed_crispr_df.columns[2:].tolist()
processed_rna_samples = processed_rna_df.columns[2:].tolist()

# get the union and intersection samples of the two lists
overlapped_crispr_rna_samples = sorted(list(set(processed_crispr_samples) & set(processed_rna_samples)))
print("The overlapped samples between protein_crispr_samples and protein_rna_samples are:")
print(len(overlapped_crispr_rna_samples))
print(overlapped_crispr_rna_samples)

# get the union set of samples of the two lists
union_crispr_rna_samples = sorted(list(set(processed_crispr_samples) | set(processed_rna_samples)))
print("The union samples between protein_crispr_samples and protein_rna_samples are:")
print(len(union_crispr_rna_samples))
print(union_crispr_rna_samples)

In [ ]:
# get the intersection based on the overlapped over omics not union over omics by overlapped_omics_annotation_samples
overlapped_omics_crispr_rna_samples = sorted(list(set(overlapped_crispr_rna_samples) & set(overlapped_omics_annotation_samples)))
print("The overlapped samples between protein_crispr_samples and protein_rna_samples are:")
print(len(overlapped_omics_crispr_rna_samples))
print(overlapped_omics_crispr_rna_samples)

# get the intersection based on the overlapped over omics not union over omics by overlapped_omics_annotation_samples for union_crispr_rna_samples
overlapped_omics_union_crispr_rna_samples = sorted(list(set(union_crispr_rna_samples) & set(overlapped_omics_annotation_samples)))
print("The overlapped samples between protein_crispr_samples and protein_rna_samples are:")
print(len(overlapped_omics_union_crispr_rna_samples))
print(overlapped_omics_union_crispr_rna_samples)

In [ ]:
# Get the index of the cell line id 'ACH-000001'
cell_line_ach_id = 'ACH-000001'
cell_line_index = sample_index_data[sample_index_data['Sample'] == cell_line_ach_id]['Index'].values[0]

# Get the values of the cell line id 'ACH-000001' in the processed_crispr_df
# Sort by ascending values and keep only the first 100 rows (lowest values)
top_pt = 10000
sample_pt_rna_df = processed_rna_df[['BMGC_PT_ID', cell_line_ach_id]].sort_values(by=cell_line_ach_id, ascending=True).head(top_pt).reset_index(drop=True)
# display the sample_pt_rna_df
display(sample_pt_rna_df)

In [ ]:
# check the overlapped pt between pt_list and t_list
overlapped_rna_pt = set(sample_pt_rna_df['BMGC_PT_ID'].tolist()).intersection(set(t_list))
print("Overlapped pt between pt_list and t_list:", overlapped_rna_pt)

### 10.3 Statisitical comparisons of p value for top 100 promoters, genes, transcripts and proteins

In [ ]:
# get the final gene, transcript, drug dataframe by filtering the overlapped_non_cancerous_samples
noncancer_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + overlapped_non_cancerous_samples]
dti_tr_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + sorted(training_cell_line_ids)]
display(noncancer_gene_df)
display(dti_tr_gene_df)
noncancer_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + overlapped_non_cancerous_samples]
dti_tr_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + sorted(training_cell_line_ids)]
display(noncancer_transcript_df)
display(dti_tr_transcript_df)
noncancer_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + overlapped_non_cancerous_samples]
dti_tr_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + sorted(training_cell_line_ids)]
display(noncancer_protein_df)
display(dti_tr_protein_df)

##### gene p values

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

def calculate_pvalues(disease_df, control_df):
    """
    Calculate p-values for each gene in each sample of disease group
    compared to the distribution of the same gene in control group using
    Wilcoxon signed-rank test.
    
    Parameters:
    disease_df: DataFrame containing disease samples
    control_df: DataFrame containing control samples
    
    Returns:
    DataFrame with same dimensions as disease_df but with p-values
    """
    # Create an empty DataFrame to store p-values with the same structure as disease_df
    pvalue_df = pd.DataFrame(index=disease_df.index, columns=disease_df.columns)
    
    # Copy the BioMedGraphica_Conn_ID column from disease_df
    pvalue_df['BioMedGraphica_Conn_ID'] = disease_df['BioMedGraphica_Conn_ID']
    
    # Get sample columns (exclude the ID column)
    disease_samples = disease_df.columns[1:]
    control_samples = control_df.columns[1:]
    
    # Print summary of what we're calculating
    print(f"Calculating p-values for {len(disease_df)} genes across {len(disease_samples)} disease samples")
    print(f"Control group: {len(control_samples)} samples")
    
    # For each gene (row) - with progress bar
    for i, gene_id in enumerate(tqdm(disease_df['BioMedGraphica_Conn_ID'], desc="Processing genes")):
        # Find the corresponding gene in control group
        control_gene_row = control_df[control_df['BioMedGraphica_Conn_ID'] == gene_id]
        
        if len(control_gene_row) == 0:
            # If gene not found in control, fill with NaN
            pvalue_df.loc[i, disease_samples] = np.nan
            continue
        
        # Get control values for this gene across all control samples
        control_values = control_gene_row.iloc[0, 1:].values.astype(float)  # Convert to float
        control_values = control_values[control_values != 0]  # Remove zeros if any
        
        if len(control_values) == 0:
            # If no valid control values, fill with NaN
            pvalue_df.loc[i, disease_samples] = np.nan
            continue
        
        # Process samples in mini-batches for better progress visibility
        batch_size = max(1, len(disease_samples) // 10)  # Process ~10 batches for better feedback
        
        # For each disease sample, calculate p-value
        for j in range(0, len(disease_samples), batch_size):
            batch_samples = disease_samples[j:j+batch_size]
            
            # Display current batch progress
            print(f"Processing batch {j//batch_size + 1}/{-(-len(disease_samples)//batch_size)} for gene {i+1}/{len(disease_df)}")
            
            for sample in batch_samples:
                # Optionally display each sample being processed
                print(f"  Processing sample: {sample}")
                
                disease_value = disease_df.loc[i, sample]
                
                # Handle special case when disease value is 0
                if disease_value == 0:
                    pvalue_df.loc[i, sample] = np.nan
                    continue
                    
                # Convert to float explicitly
                disease_value = float(disease_value)
                
                # Calculate p-value using Wilcoxon signed-rank test
                try:
                    # For Wilcoxon test, we need to compare the distribution to the specific value
                    # Create an array of differences between each control value and the disease value
                    differences = control_values - disease_value
                    
                    # Use Wilcoxon signed-rank test (tests if differences are centered around 0)
                    stat, p_value = stats.wilcoxon(differences)
                    pvalue_df.loc[i, sample] = p_value
                except (TypeError, ValueError) as e:
                    # Wilcoxon test requires at least 1 non-zero difference
                    # If all differences are zero or there are too few samples, we'll get an error
                    # print(f"Error in Wilcoxon test for gene {gene_id}, sample {sample}: {e}")
                    pvalue_df.loc[i, sample] = np.nan
                
        # Show a progress update for every 5% of genes
        if i % max(1, len(disease_df) // 20) == 0 and i > 0:
            completed = 100 * i / len(disease_df)
            print(f"Completed {completed:.1f}% ({i}/{len(disease_df)} genes)")
    
    return pvalue_df

# # Apply the function to calculate p-values
# print("Starting p-value calculation...")
# dti_tr_gene1_df = dti_tr_gene_df[['BioMedGraphica_Conn_ID', 'ACH-000001']].copy()
# pvalue_df = calculate_pvalues(dti_tr_gene1_df, noncancer_gene_df)

# # Display the first few rows of the result
# print("\nP-value calculation complete. Here's a preview of the results:")
# print(pvalue_df.head())

In [ ]:
# # Filter out the NaN values and get the lowest p-values
# pvalue_c_df = pvalue_df.dropna(subset=['ACH-000001'])
# pvalue_c_df = pvalue_c_df.sort_values(by='ACH-000001').reset_index(drop=True)

# # Reset the display format to scientific notation
# pd.reset_option('display.float_format')
# # Set scientific notation with reasonable precision
# pd.set_option('display.float_format', lambda x: '{:.2e}'.format(x) if abs(x) < 0.01 else '{:.4f}'.format(x))
# display(pvalue_c_df)

In [ ]:
# # Annotate pvalue_df with gene HGNC_Symbol by bmgc_gene_df
# bmgc_gene_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
# # keep BioMedGraphica_ID and HGNC_Symbol
# bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
# # merge the biomedgraphica_id with the pvalue_c_df
# pvalue_hgnc_df = pd.merge(pvalue_c_df, bmgc_gene_df, on='BioMedGraphica_Conn_ID', how='left')
# # drop the BioMedGraphica_Conn_ID column and drop NaN values in HGNC_Symbol column
# pvalue_hgnc_df = pvalue_hgnc_df.drop(columns=['BioMedGraphica_Conn_ID']).reset_index(drop=True)
# # display the pvalue_df
# display(pvalue_hgnc_df)

In [ ]:
def extract_pvalue_answer(pvalue_hgnc_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm=100):
    # Convert the column to numeric type, forcing errors to become NaN
    pvalue_hgnc_df[sample_ach_name] = pd.to_numeric(pvalue_hgnc_df[sample_ach_name], errors='coerce')
    # Extract the lowest top_bm values for the given sample name
    top_bm_genes = pvalue_hgnc_df.nsmallest(top_bm, sample_ach_name)[['HGNC_Symbol', sample_ach_name]]
    # Sort the top bm genes by their values in ascending order
    top_bm_genes = top_bm_genes.sort_values(by=sample_ach_name, ascending=True).reset_index(drop=True)
    top_pvalue_bm_gene_hgnc_name_list = top_bm_genes['HGNC_Symbol'].tolist()
    # Merge with the bmgc_gene_df to get the BioMedGraphica_Conn_ID
    bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_bm_bmgc_gene_df = pd.merge(bmgc_gene_df, top_bm_genes, left_on='HGNC_Symbol', right_on='HGNC_Symbol', how='inner')
    # Get the corresponding proteins
    top_bm_bmgc_gene_protein_df = pd.merge(gene_transcript_protein_entity_df, top_bm_bmgc_gene_df, 
                                          left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', 
                                          how='inner')
    # Remove the BioMedGraphica_Conn_ID column
    if 'BioMedGraphica_Conn_ID' in top_bm_bmgc_gene_protein_df:
        top_bm_bmgc_gene_protein_df = top_bm_bmgc_gene_protein_df.drop(columns=['BioMedGraphica_Conn_ID'])
    # Get additional protein information
    top_bm_bmgc_gene_protein_info_df = pd.merge(top_bm_bmgc_gene_protein_df, bmgc_protein_llmnameid_combined_df, 
                                               left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', 
                                               how='inner')
    # Drop unnecessary columns
    columns_to_drop = ['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID']
    if sample_ach_name in top_bm_bmgc_gene_protein_info_df.columns:
        columns_to_drop.append(sample_ach_name)
    
    top_bm_bmgc_gene_protein_info_df = top_bm_bmgc_gene_protein_info_df.drop(columns=columns_to_drop, errors='ignore')
    
    # Extract the lists of protein IDs and names
    top_bm_gene_protein_bmgc_id_list = top_bm_bmgc_gene_protein_info_df['BMGC_PT_ID'].tolist()
    top_bm_gene_protein_bmgc_llmnameid_list = top_bm_bmgc_gene_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    
    return top_pvalue_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list

# # Example usage
# sample_ach_name = 'ACH-000001'
# top_bm = 100
# bmgc_protein_llmnameid_combined_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
# top_pvalue_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list = extract_pvalue_answer(pvalue_hgnc_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm)
# print(f"Top {top_bm} Gene HGNC Names:", top_pvalue_bm_gene_hgnc_name_list)
# print(f"Top {top_bm} Gene Protein BMGC IDs:", top_bm_gene_protein_bmgc_id_list)
# print(f"Top {top_bm} Gene Protein BMGC LLM Name IDs:", top_bm_gene_protein_bmgc_llmnameid_list)
# # Save the [top_pvalue_bm_gene_hgnc_name_list] to a csv file
# top_pvalue_bm_gene_hgnc_name_df = pd.DataFrame(top_pvalue_bm_gene_hgnc_name_list, columns=['HGNC_Symbol'])
# # Make dir for P value data
# import os
# if not os.path.exists('../pvalues/'):
#     os.makedirs('../pvalues/')
# top_pvalue_bm_gene_hgnc_name_df.to_csv('../pvalues/top_pvalue_bm_gene_hgnc_name.csv', index=True, header=True)

## 11. LLM QA

### 11.0 Formulate the samples

In [ ]:
training_cell_line_ids = list(set(final_training_drug_screen_df['depMapID']))
dti_training_samples = sorted(training_cell_line_ids)
print("The training samples are:", len(dti_training_samples))

In [ ]:
current_crispr_samples = sorted(list(set(processed_crispr_df.columns[2:])))
print("The current crispr samples are:", len(current_crispr_samples))
current_rna_samples = sorted(list(set(processed_rna_df.columns[2:])))
print("The current rna samples are:", len(current_rna_samples))

overlapped_crispr_rna_samples = sorted(list(set(current_crispr_samples) & set(current_rna_samples)))
print("The overlapped samples between current_crispr_samples and current_rna_samples are:", len(overlapped_crispr_rna_samples))

# overlapped samples between dti_training_samples and overlapped_crispr_rna_samples
overlapped_dti_crispr_rna_samples = sorted(list(set(dti_training_samples) & set(overlapped_crispr_rna_samples)))
print("The overlapped samples between dti_training_samples and overlapped_crispr_rna_samples are:", len(overlapped_dti_crispr_rna_samples))

In [ ]:
overlapped_dti_crispr_rna_samples[0]

In [ ]:
import random
import numpy as np

# Fix all random seeds
random.seed(42)
np.random.seed(42)  # Also set NumPy seed if you're using it elsewhere

# Make a copy of the list before shuffling to avoid modifying the original
samples_to_shuffle = overlapped_dti_crispr_rna_samples.copy()
random.shuffle(samples_to_shuffle)

# Split into train and test sets
split_ratio = 0.8
split_index = int(len(samples_to_shuffle) * split_ratio)
train_dti_crispr_rna_samples = sorted(samples_to_shuffle[:split_index])
test_dti_crispr_rna_samples = sorted(samples_to_shuffle[split_index:])

print("The training samples are:", len(train_dti_crispr_rna_samples))
print(train_dti_crispr_rna_samples[:10])
print("The testing samples are:", len(test_dti_crispr_rna_samples))
print(test_dti_crispr_rna_samples[:10])

# Save the train and test samples to a csv file
train_dti_crispr_rna_samples_df = pd.DataFrame(train_dti_crispr_rna_samples, columns=['Sample'])
train_dti_crispr_rna_samples_df.to_csv('../QA_Data/train_dti_crispr_rna_samples.csv', index=False, header=True)
test_dti_crispr_rna_samples_df = pd.DataFrame(test_dti_crispr_rna_samples, columns=['Sample'])
test_dti_crispr_rna_samples_df.to_csv('../QA_Data/test_dti_crispr_rna_samples.csv', index=False, header=True)

### 11.1 CRISPR-QA

#### 11.1.1 Cell Line and Disease Info

In [ ]:
# Cell line information
dti_sample_index = pd.read_csv('../BMG/DTI_data/sample_index.csv')
dti_sample_info = pd.read_csv('../BMG/process_data/dti_combined_samples.csv')
dti_sample_info_index = pd.merge(dti_sample_info, dti_sample_index, how='inner', left_on='depMapID', right_on='Sample').drop(columns=['Sample'])
# Filter out the samples that are in the overlapped_dti_crispr_rna_samples
dti_sample_info_index = dti_sample_info_index[dti_sample_info_index['depMapID'].isin(overlapped_dti_crispr_rna_samples)].reset_index(drop=True)
# Replace " | " with " or " in the BMGC_Disease_name column
dti_sample_info_index['BMGC_Disease_name'] = dti_sample_info_index['BMGC_Disease_name'].replace(r' \| ', ' or ', regex=True)
# Only keep the columns ['depMapID', 'Name', 'BMGC_Disease_ID', 'BMGC_Disease_name', 'Index']
dti_sample_info_index = dti_sample_info_index[['depMapID', 'Name', 'BMGC_Disease_ID', 'BMGC_Disease_name', 'Index']].reset_index(drop=True)
display(dti_sample_info_index)

#### 11.1.2 Omics Info

##### Methylation

In [ ]:
mapped_methy_df = methy_df.rename(columns=methy_map_dict)
promoter_overlapped_dti_crispr_rna_samples = sorted(list(set(mapped_methy_df.columns[1:]) & set(overlapped_dti_crispr_rna_samples)))
dti_crispr_rna_methy_df = mapped_methy_df[['locus_id'] + promoter_overlapped_dti_crispr_rna_samples].copy()
display(dti_crispr_rna_methy_df)

In [ ]:
bmgc_promoter_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
bmgc_protein_llmnameid_combined_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
display(bmgc_protein_llmnameid_combined_df)

In [ ]:
promoter_gene_df = pd.concat([bmgc_promoter_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_PM_ID'}), bmgc_gene_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_GN_ID'})], axis=1)
promoter_protein_entity_df = pd.merge(promoter_gene_transcript_protein_entity_df, promoter_gene_df, left_on='BMGC_GN_ID', right_on='BMGC_GN_ID', how='left').drop(columns=['BMGC_GN_ID'])
promoter_protein_entity_df = promoter_protein_entity_df[['BMGC_PM_ID', 'BMGC_PT_ID']]
display(promoter_protein_entity_df)

In [ ]:
def extract_pm_info(dti_crispr_rna_methy_df, bmgc_promoter_df, promoter_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Extract the top k highest values for the given sample name
    top_k_promoters = dti_crispr_rna_methy_df.nlargest(k, sample_ach_name)[['locus_id', sample_ach_name]]
    # Sort the top k promoters by their values in descending order
    top_k_promoters = top_k_promoters.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_promoter_name_list = top_k_promoters['locus_id'].tolist()
    display(top_k_promoters)
    # Merge with the bmgc_promoter_df to get the BioMedGraphica_Conn_ID
    bmgc_promoter_df = bmgc_promoter_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_promoter_df = pd.merge(bmgc_promoter_df, top_k_promoters, left_on='HGNC_Symbol', right_on='locus_id', how='inner')
    # Get the corresponding proteins
    top_k_bmgc_promoter_protein_df = pd.merge(promoter_protein_entity_df, top_k_bmgc_promoter_df, left_on='BMGC_PM_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'locus_id'])
    top_k_bmgc_promoter_protein_info_df = pd.merge(top_k_bmgc_promoter_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_k_promoter_protein_bmgc_id_list = top_k_bmgc_promoter_protein_info_df['BMGC_PT_ID'].tolist()
    display(top_k_bmgc_promoter_protein_info_df)
    top_k_promoter_protein_bmgc_llmnameid_list = top_k_bmgc_promoter_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_k_promoter_name_list, top_k_promoter_protein_bmgc_id_list, top_k_promoter_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
k=100
top_k_promoter_name_list, top_k_promoter_protein_bmgc_id_list, top_k_promoter_protein_bmgc_llmnameid_list = extract_pm_info(dti_crispr_rna_methy_df, bmgc_promoter_df, promoter_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Promoter Names:", top_k_promoter_name_list)
print(f"Top {k} Promoter Protein BMGC IDs:", top_k_promoter_protein_bmgc_id_list)
print(f"Top {k} Promoter Protein BMGC LLM Name IDs:", top_k_promoter_protein_bmgc_llmnameid_list)


##### Gene Mutation

In [ ]:
# select the columns in the overlapped_dti_crispr_rna_samples for gene_df
dti_crispr_rna_gene_df = gene_df[['gene_name'] + sorted(overlapped_dti_crispr_rna_samples)].copy()
display(dti_crispr_rna_gene_df)

In [ ]:
bmgc_gene_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
bmgc_protein_llmnameid_combined_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
display(bmgc_protein_llmnameid_combined_df)

In [ ]:
def extract_gn_info(dti_crispr_rna_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_crispr_rna_gene_df columns
    if sample_ach_name not in dti_crispr_rna_gene_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_genes = dti_crispr_rna_gene_df.nlargest(k, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top k genes by their values in descending order
    top_k_genes = top_k_genes.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_gene_hgnc_name_list = top_k_genes['gene_name'].tolist()
    # Merge with the bmgc_gene_df to get the BioMedGraphica_Conn_ID
    bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_gene_df = pd.merge(bmgc_gene_df, top_k_genes, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_k_bmgc_gene_protein_df = pd.merge(gene_transcript_protein_entity_df, top_k_bmgc_gene_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_k_bmgc_gene_protein_info_df = pd.merge(top_k_bmgc_gene_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_k_gene_protein_bmgc_id_list = top_k_bmgc_gene_protein_info_df['BMGC_PT_ID'].tolist()
    top_k_gene_protein_bmgc_llmnameid_list = top_k_bmgc_gene_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
k=10
top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list = extract_gn_info(dti_crispr_rna_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Gene HGNC Names:", top_k_gene_hgnc_name_list)
print(f"Top {k} Gene Protein BMGC IDs:", top_k_gene_protein_bmgc_id_list)
print(f"Top {k} Gene Protein BMGC LLM Name IDs:", top_k_gene_protein_bmgc_llmnameid_list)

##### Transcript Exp

In [ ]:
# select the columns in the overlapped_dti_crispr_rna_samples for raw_transcript_df
transcript_overlapped_dti_crispr_rna_samples = sorted(list(set(raw_transcript_df.columns[1:]) & set(overlapped_dti_crispr_rna_samples)))
dti_crispr_rna_transcript_df = raw_transcript_df[['gene_name'] + transcript_overlapped_dti_crispr_rna_samples].copy()
display(dti_crispr_rna_transcript_df)

In [ ]:
bmgc_transcript_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')

In [ ]:
def extract_ts_info(dti_crispr_rna_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_crispr_rna_transcript_df columns
    if sample_ach_name not in dti_crispr_rna_transcript_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_transcripts = dti_crispr_rna_transcript_df.nlargest(k, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top k transcripts by their values in descending order
    top_k_transcripts = top_k_transcripts.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_transcript_hgnc_name_list = top_k_transcripts['gene_name'].tolist()
    # Merge with the bmgc_transcript_df to get the BioMedGraphica_Conn_ID
    bmgc_transcript_df = bmgc_transcript_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_transcript_df = pd.merge(bmgc_transcript_df, top_k_transcripts, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_k_bmgc_transcript_protein_df = pd.merge(transcript_protein_entity_df, top_k_bmgc_transcript_df, left_on='BMGC_TS_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_k_bmgc_transcript_protein_info_df = pd.merge(top_k_bmgc_transcript_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_k_transcript_protein_bmgc_id_list = top_k_bmgc_transcript_protein_info_df['BMGC_PT_ID'].tolist()
    top_k_transcript_protein_bmgc_llmnameid_list = top_k_bmgc_transcript_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
k=10
top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list = extract_ts_info(dti_crispr_rna_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Transcript HGNC Names:", top_k_transcript_hgnc_name_list)
print(f"Top {k} Transcript Protein BMGC IDs:", top_k_transcript_protein_bmgc_id_list)
print(f"Top {k} Transcript Protein BMGC LLM Name IDs:", top_k_transcript_protein_bmgc_llmnameid_list)

##### Protein Exp

In [ ]:
bmg_protein_all_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')
bmg_protein_all_df = bmg_protein_all_df[['BioMedGraphica_Conn_ID', 'Uniprot_ID', 'HGNC_Symbol']].copy()
display(bmg_protein_all_df)

In [ ]:
# Rename columns in raw_protein_df using the provided mapping
raw_protein_map_df = raw_protein_df.rename(columns=protein_map_dict)
# Merge with bmg_protein_all_df to get HGNC symbols
symbol_protein_map_df = pd.merge(raw_protein_map_df, bmg_protein_all_df, left_on='Uniprot_Acc', right_on='Uniprot_ID', how='inner')
# Reorder columns: keep Uniprot IDs and protein expression values
symbol_protein_map_df = symbol_protein_map_df[['Uniprot_ID', 'Uniprot_Acc', 'HGNC_Symbol'] + sorted(set(symbol_protein_map_df.columns) - {'Uniprot_ID', 'Uniprot_Acc', 'HGNC_Symbol'})]
# Identify overlapping samples between protein data and the provided sample list
protein_overlapped_samples = sorted(set(symbol_protein_map_df.columns) & set(overlapped_dti_crispr_rna_samples))
# Select only HGNC symbol and overlapping sample columns
dti_crispr_rna_protein_df = symbol_protein_map_df[['HGNC_Symbol'] + protein_overlapped_samples].copy()
# Split multiple HGNC symbols by ";" and expand into multiple rows
dti_crispr_rna_protein_df = dti_crispr_rna_protein_df.assign(HGNC_Symbol=dti_crispr_rna_protein_df['HGNC_Symbol'].str.split(';')).explode('HGNC_Symbol')

# Remove leading/trailing whitespace in gene symbols
dti_crispr_rna_protein_df['HGNC_Symbol'] = dti_crispr_rna_protein_df['HGNC_Symbol'].str.strip()
# Drop rows with empty or missing gene symbols
dti_crispr_rna_protein_df = dti_crispr_rna_protein_df[dti_crispr_rna_protein_df['HGNC_Symbol'].notna() & (dti_crispr_rna_protein_df['HGNC_Symbol'] != '')].reset_index(drop=True)

# Display final dataframe
display(dti_crispr_rna_protein_df)


In [ ]:
bmgc_protein_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')
display(bmgc_protein_df)

In [ ]:
def extract_pt_info(dti_crispr_rna_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_crispr_rna_protein_df columns
    if sample_ach_name not in dti_crispr_rna_protein_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_proteins = dti_crispr_rna_protein_df.nlargest(k, sample_ach_name)[['HGNC_Symbol', sample_ach_name]]
    # Sort the top k proteins by their values in descending order
    top_k_proteins = top_k_proteins.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_protein_hgnc_name_list = top_k_proteins['HGNC_Symbol'].tolist()
    # Merge with the bmgc_protein_df to get the BioMedGraphica_Conn_ID
    bmgc_protein_df = bmgc_protein_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_protein_df = pd.merge(bmgc_protein_df, top_k_proteins, left_on='HGNC_Symbol', right_on='HGNC_Symbol', how='inner')
    # Get the corresponding protein information
    top_k_bmgc_protein_info_df = pd.merge(top_k_bmgc_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_ID', sample_ach_name])
    top_k_protein_bmgc_id_list = top_k_bmgc_protein_info_df['BioMedGraphica_Conn_ID'].tolist()
    # Replace both "|" and ";" with " or "
    top_k_protein_bmgc_llmnameid_list = top_k_bmgc_protein_info_df['Names_and_IDs'].replace([r' \| ', r';'], ' or ', regex=True).tolist()
    return top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
k = 10
top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list = extract_pt_info(dti_crispr_rna_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Protein HGNC Names:", top_k_protein_hgnc_name_list)
print(f"Top {k} Protein BMGC IDs:", top_k_protein_bmgc_id_list)
print(f"Top {k} Protein BMGC LLM Name IDs:", top_k_protein_bmgc_llmnameid_list)

#### 11.1.3 Related Proteins

In [ ]:
def extract_disease_protein(selected_sample_disease_bmgc_id, dti_edge_index_path, 
                    omics_node_index_df, nodeid_index_dict, index_nodeid_dict):
    """
    Extract protein directly connected to a disease node.
    
    Args:
        sample_disease_bmgc_id_index (int): Index of the disease node
        dti_edge_index_path (str): Path to the DTI edge index file
        omics_node_index_df (pd.DataFrame): DataFrame with node type information
        
    Returns:
        tuple: (disease_protein_index, disease_protein_bmgc_id)
    """
    # Extract the index based on the selected disease BMGC ID
    sample_disease_bmgc_id_index = nodeid_index_dict[selected_sample_disease_bmgc_id]
    
    # Load DTI edge index
    dti_all_edge_index = np.load(dti_edge_index_path)
    
    # Find incoming edges (source nodes that point to the disease)
    incoming_mask = dti_all_edge_index[1, :] == sample_disease_bmgc_id_index
    incoming_source_nodes = dti_all_edge_index[0, incoming_mask]
    
    # Find outgoing edges (target nodes that the disease points to)
    outgoing_mask = dti_all_edge_index[0, :] == sample_disease_bmgc_id_index
    outgoing_target_nodes = dti_all_edge_index[1, outgoing_mask]
    
    # Combine all neighbor nodes (both incoming and outgoing)
    disease_related_nodes = np.concatenate([incoming_source_nodes, outgoing_target_nodes])
    unique_disease_related_nodes = np.unique(disease_related_nodes)
    
    # Get protein node index
    protein_node_index_df = omics_node_index_df[omics_node_index_df['Type'] == 'Protein']
    protein_node_index_list = protein_node_index_df['Index'].tolist()
    
    # Filter to get only protein nodes directly connected to the disease
    disease_protein_index = sorted(
        list(set(unique_disease_related_nodes) & set(protein_node_index_list))
    )
    
    # Map protein index to BMGC id
    disease_protein_bmgc_id = [index_nodeid_dict[i] for i in disease_protein_index]
    
    return disease_protein_index, disease_protein_bmgc_id


def extract_ppi_nodes(disease_protein_index, dti_edge_index_path, omics_node_index_df, index_nodeid_dict):
    """
    Extract protein-protein interaction (PPI) nodes that interact with disease-related protein.
    
    Args:
        disease_protein_index (list): List of protein node index directly related to the disease
        dti_edge_index_path (str): Path to the DTI edge index file
        omics_node_index_df (pd.DataFrame): DataFrame with node type information
        
    Returns:
        tuple: (ppi_nodes_index, ppi_nodes_bmgc_id)
    """

    # Load DTI edge index
    dti_all_edge_index = np.load(dti_edge_index_path)
    
    # Get protein node index
    protein_node_index_df = omics_node_index_df[omics_node_index_df['Type'] == 'Protein']
    protein_node_index_list = protein_node_index_df['Index'].tolist()
    
    # Get all nodes related to the identified protein neighbors (second hop)
    protein_related_nodes = []
    
    # Iterate through each protein neighbor node index
    for protein_node_idx in disease_protein_index:
        # Find incoming edges (nodes that point to this protein)
        protein_incoming_mask = dti_all_edge_index[1, :] == protein_node_idx
        protein_incoming_sources = dti_all_edge_index[0, protein_incoming_mask]
        
        # Find outgoing edges (nodes that this protein points to)
        protein_outgoing_mask = dti_all_edge_index[0, :] == protein_node_idx
        protein_outgoing_targets = dti_all_edge_index[1, protein_outgoing_mask]
        
        # Add these connected nodes to our list
        protein_related_nodes.extend(protein_incoming_sources)
        protein_related_nodes.extend(protein_outgoing_targets)
    
    # Convert to numpy array and get unique nodes
    protein_related_nodes = np.array(protein_related_nodes)
    unique_protein_related_nodes = np.unique(protein_related_nodes)
    
    # Remove any protein nodes themselves from this list to avoid duplication
    unique_protein_related_nodes = np.setdiff1d(
        unique_protein_related_nodes, disease_protein_index
    )
    
    # Filter to only keep protein nodes among the second-hop neighbors
    ppi_nodes_index = sorted(
        list(set(unique_protein_related_nodes) & set(protein_node_index_list))
    )
    
    # Map PPI node index to BMGC id
    ppi_nodes_bmgc_id = [index_nodeid_dict[i] for i in ppi_nodes_index]

    return ppi_nodes_index, ppi_nodes_bmgc_id

In [ ]:
# Mapping dictionary between ID and Index
nodeid_index_data = pd.read_csv('../BMG/DTI_data/nodes_index.csv')
nodeid_index_dict = dict(zip(nodeid_index_data['Node'], nodeid_index_data['Index']))
index_nodeid_dict = dict(zip(nodeid_index_data['Index'], nodeid_index_data['Node']))
# use omics_node_index_df to get the node type
omics_node_index_df = pd.read_csv('../BMG/Pretrain_data/omics_nodes_index.csv')

In [ ]:
# Rename columns in raw_protein_df using the provided mapping
raw_protein_map_df = raw_protein_df.rename(columns=protein_map_dict)
# Merge with bmg_protein_all_df to get HGNC symbols
symbol_protein_map_df = pd.merge(raw_protein_map_df, bmg_protein_all_df, left_on='Uniprot_Acc', right_on='Uniprot_ID', how='inner')
# Reorder columns: keep Uniprot IDs and protein expression values
symbol_protein_map_df = symbol_protein_map_df[['Uniprot_ID', 'Uniprot_Acc', 'HGNC_Symbol'] + sorted(set(symbol_protein_map_df.columns) - {'Uniprot_ID', 'Uniprot_Acc', 'HGNC_Symbol'})]
# Identify overlapping samples between protein data and the provided sample list
protein_overlapped_samples = sorted(set(symbol_protein_map_df.columns) & set(overlapped_dti_crispr_rna_samples))
# Select only HGNC symbol and overlapping sample columns
dti_crispr_rna_protein_df = symbol_protein_map_df[['HGNC_Symbol'] + protein_overlapped_samples].copy()
# Split multiple HGNC symbols by ";" and expand into multiple rows
dti_crispr_rna_protein_df = dti_crispr_rna_protein_df.assign(HGNC_Symbol=dti_crispr_rna_protein_df['HGNC_Symbol'].str.split(';')).explode('HGNC_Symbol')

# Remove leading/trailing whitespace in gene symbols
dti_crispr_rna_protein_df['HGNC_Symbol'] = dti_crispr_rna_protein_df['HGNC_Symbol'].str.strip()
# Drop rows with empty or missing gene symbols
dti_crispr_rna_protein_df = dti_crispr_rna_protein_df[dti_crispr_rna_protein_df['HGNC_Symbol'].notna() & (dti_crispr_rna_protein_df['HGNC_Symbol'] != '')].reset_index(drop=True)

# Display final dataframe
display(dti_crispr_rna_protein_df)


In [ ]:
def bmgc_pt_id_to_hgnc(bmgc_id_list, bmgc_protein_df):
    """
    Convert a list of BioMedGraphica IDs to their corresponding HGNC symbols.
    
    Args:
        bmgc_id_list (list): List of BioMedGraphica IDs
        bmgc_protein_df (pd.DataFrame): DataFrame with BioMedGraphica IDs and HGNC symbols
        
    Returns:
        tuple: (
            dict: Dictionary mapping each BioMedGraphica ID to its list of HGNC symbols,
            list: Combined list of all HGNC symbols
        )
    """
    # Ensure bmgc_id_list is actually a list
    if not isinstance(bmgc_id_list, list):
        bmgc_id_list = [bmgc_id_list]
    
    results = {}
    all_hgnc_symbols = []
    
    for bmgc_id in bmgc_id_list:
        # Filter the DataFrame for the given BioMedGraphica ID
        filtered_df = bmgc_protein_df[bmgc_protein_df['BioMedGraphica_Conn_ID'] == bmgc_id]
        
        # Skip if no match found
        if filtered_df.empty:
            results[bmgc_id] = []
            continue
            
        # Get the HGNC symbols
        hgnc_value = filtered_df['HGNC_Symbol'].values[0]
        
        # Skip if HGNC symbol is NaN
        if pd.isna(hgnc_value):
            results[bmgc_id] = []
            continue
        
        # Process valid HGNC symbols
        hgnc_list = list(set(hgnc_value.split(';')))
        hgnc_list = [hgnc.strip() for hgnc in hgnc_list if hgnc.strip() != '']
        
        results[bmgc_id] = hgnc_list
        all_hgnc_symbols.extend(hgnc_list)
    
    # Remove duplicates from the combined list
    all_hgnc_symbols = list(set(all_hgnc_symbols))
    
    return results, all_hgnc_symbols

# Example usage
bmgc_ids = ['BMGC_PT000001', 'BMGC_PT013541']
hgnc_dict, all_hgnc_symbols = bmgc_pt_id_to_hgnc(bmgc_ids, bmgc_protein_df)
print(hgnc_dict)
print(all_hgnc_symbols)


In [ ]:
def hgnc_to_bmgc_pt_id(hgnc_list, bmgc_protein_df):
    """
    Convert a list of HGNC symbols to their corresponding BioMedGraphica IDs.
    
    Args:
        hgnc_list (list): List of HGNC symbols
        bmgc_protein_df (pd.DataFrame): DataFrame with BioMedGraphica IDs and HGNC symbols
        
    Returns:
        tuple: (
            dict: Dictionary mapping each HGNC symbol to its list of BioMedGraphica IDs,
            list: Combined list of all BioMedGraphica IDs
        )
    """
    # Ensure hgnc_list is actually a list
    if not isinstance(hgnc_list, list):
        hgnc_list = [hgnc_list]
    
    results = {}
    all_bmgc_ids = []
    
    for hgnc in hgnc_list:
        # Filter the DataFrame for the given HGNC symbol
        filtered_df = bmgc_protein_df[bmgc_protein_df['HGNC_Symbol'] == hgnc]
        
        # Skip if no match found
        if filtered_df.empty:
            results[hgnc] = []
            continue
        
        # Get the BioMedGraphica IDs
        bmgc_value = filtered_df['BioMedGraphica_Conn_ID'].values[0]
        
        # Skip if BioMedGraphica ID is NaN
        if pd.isna(bmgc_value):
            results[hgnc] = []
            continue
        
        # Process valid BioMedGraphica IDs
        bmgc_list = list(set(bmgc_value.split(';')))
        bmgc_list = [bmgc.strip() for bmgc in bmgc_list if bmgc.strip() != '']
        
        results[hgnc] = bmgc_list
        all_bmgc_ids.extend(bmgc_list)
    
    # Remove duplicates from the combined list
    all_bmgc_ids = list(set(all_bmgc_ids))
    
    return results, all_bmgc_ids

# Example usage
hgnc_list = ['BRCA1', 'TP53']
bmgc_dict, all_bmgc_ids = hgnc_to_bmgc_pt_id(hgnc_list, bmgc_protein_df)
print(bmgc_dict)
print(all_bmgc_ids)

In [ ]:
def extract_kg_related_proteins(selected_sample_disease_bmgc_id, dti_edge_index_path,  # '../BMG/DTI_data/edge_index.npy'
                               omics_node_index_df, nodeid_index_dict, index_nodeid_dict):
    """
    Extract disease-related proteins and their interactions from the knowledge graph.
    
    Args:
        selected_sample_disease_bmgc_id (str): BMGC ID of the selected disease
        dti_edge_index_path (str): Path to the DTI edge index file
        omics_node_index_df (pd.DataFrame): DataFrame with node type information
        nodeid_index_dict (dict): Mapping from node ID to index
        index_nodeid_dict (dict): Mapping from index to node ID
    
    Returns:
        tuple: (disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id)
    """
    
    # Extract disease-protein connections
    disease_protein_index, disease_protein_bmgc_id = extract_disease_protein(
        selected_sample_disease_bmgc_id=selected_sample_disease_bmgc_id,
        dti_edge_index_path=dti_edge_index_path,
        omics_node_index_df=omics_node_index_df,
        nodeid_index_dict=nodeid_index_dict,
        index_nodeid_dict=index_nodeid_dict
    )

    # Extract protein-protein interactions (can replace this with LLM to generate PPI, may need NER and mapping to BMGC id)
    ppi_nodes_index, ppi_nodes_bmgc_id = extract_ppi_nodes(
        disease_protein_index=disease_protein_index,
        dti_edge_index_path=dti_edge_index_path,
        omics_node_index_df=omics_node_index_df,
        index_nodeid_dict=index_nodeid_dict
    )

    # Convert the BMGC id into HGNC symbol

    return disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id

# Example usage
# selected_sample_disease_bmgc_id = 'BMGC_DS16646'
selected_sample_disease_bmgc_id = 'BMGC_DS07934'
dti_edge_index_path = '../BMG/DTI_data/edge_index.npy'

disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id = extract_kg_related_proteins(
    selected_sample_disease_bmgc_id=selected_sample_disease_bmgc_id,
    dti_edge_index_path=dti_edge_index_path,
    omics_node_index_df=omics_node_index_df,
    nodeid_index_dict=nodeid_index_dict,
    index_nodeid_dict=index_nodeid_dict
)
print("Disease Protein Index:", disease_protein_index)
print("Disease Protein BMGC ID:", disease_protein_bmgc_id)

#### 11.1.4 CRISPR Answer

In [ ]:
answer_crispr_df = crispr_df[['gene_name'] + sorted(overlapped_dti_crispr_rna_samples)].copy()
display(answer_crispr_df)

In [ ]:
def extract_answer(answer_crispr_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm=100):
    # Extract the lowest top_bm values for the given sample name
    top_bm_genes = answer_crispr_df.nsmallest(top_bm, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top bm genes by their values in ascending order
    top_bm_genes = top_bm_genes.sort_values(by=sample_ach_name, ascending=True).reset_index(drop=True)
    top_bm_gene_hgnc_name_list = top_bm_genes['gene_name'].tolist()
    # Merge with the bmgc_gene_df to get the BioMedGraphica_Conn_ID
    bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_bm_bmgc_gene_df = pd.merge(bmgc_gene_df, top_bm_genes, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_bm_bmgc_gene_protein_df = pd.merge(gene_transcript_protein_entity_df, top_bm_bmgc_gene_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_bm_bmgc_gene_protein_info_df = pd.merge(top_bm_bmgc_gene_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_bm_gene_protein_bmgc_id_list = top_bm_bmgc_gene_protein_info_df['BMGC_PT_ID'].tolist()
    top_bm_gene_protein_bmgc_llmnameid_list = top_bm_bmgc_gene_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
top_bm = 1000
bmgc_protein_llmnameid_combined_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list = extract_answer(answer_crispr_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm)
print(f"Top {top_bm} Gene HGNC Names:", top_bm_gene_hgnc_name_list)
print(f"Top {top_bm} Gene Protein BMGC IDs:", top_bm_gene_protein_bmgc_id_list)
print(f"Top {top_bm} Gene Protein BMGC LLM Name IDs:", top_bm_gene_protein_bmgc_llmnameid_list)

##### Check the overlap between CRISPR and pvalue answer

In [ ]:
# # check the overlap elements/genes between top_pvalue_bm_gene_hgnc_name_list and top_bm_gene_hgnc_name_list
# overlap_genes = list(set(top_bm_gene_hgnc_name_list) & set(top_pvalue_bm_gene_hgnc_name_list))
# # Count the number of overlapping genes
# overlap_count = len(overlap_genes)
# # Print the overlapping genes and their count
# print(f"Number of overlapping genes: {overlap_count}")
# print("Overlap Genes:", overlap_genes)

#### 11.1.5 KG Info

In [ ]:
display(bmgc_protein_df)
display(bmgc_relation_df)

In [ ]:
def find_protein_relationships(hgnc_symbols, bmgc_protein_df, bmgc_relation_df):
    """
    Find relationships between a list of proteins based on HGNC symbols.
    
    Parameters:
    -----------
    hgnc_symbols : list
        List of HGNC symbols to find relationships between
    bmgc_protein_df : pandas DataFrame
        DataFrame containing BioMedGraphica_Conn_ID and HGNC_Symbol columns
    bmgc_relation_df : pandas DataFrame
        DataFrame containing BMGC_From_ID and BMGC_To_ID columns
        
    Returns:
    --------
    tuple (pandas DataFrame, list)
        - DataFrame with source_symbol, target_symbol and their relationship
        - List of text descriptions of relationships in "A -> B" format
    """
    # Filter the protein DataFrame to only include the proteins we care about
    filtered_proteins = bmgc_protein_df[bmgc_protein_df['HGNC_Symbol'].isin(hgnc_symbols)]
    
    # Create a mapping from HGNC symbol to BMGC ID
    hgnc_to_bmgc = dict(zip(filtered_proteins['HGNC_Symbol'], filtered_proteins['BioMedGraphica_Conn_ID']))
    bmgc_to_hgnc = dict(zip(filtered_proteins['BioMedGraphica_Conn_ID'], filtered_proteins['HGNC_Symbol']))
    
    # Get all BMGC IDs of our proteins
    bmgc_ids = list(hgnc_to_bmgc.values())
    
    # Filter the relationship DataFrame to only include relationships between our proteins
    protein_relations = bmgc_relation_df[
        bmgc_relation_df['BMGC_From_ID'].isin(bmgc_ids) & 
        bmgc_relation_df['BMGC_To_ID'].isin(bmgc_ids)
    ]
    
    # Map the BMGC IDs back to HGNC symbols
    result_data = []
    text_descriptions = []
    
    for _, row in protein_relations.iterrows():
        source_bmgc = row['BMGC_From_ID']
        target_bmgc = row['BMGC_To_ID']
        
        if source_bmgc in bmgc_to_hgnc and target_bmgc in bmgc_to_hgnc:
            source_symbol = bmgc_to_hgnc[source_bmgc]
            target_symbol = bmgc_to_hgnc[target_bmgc]
            
            # Create text description
            text_description = f"{source_symbol} -> {target_symbol}"
            text_descriptions.append(text_description)
            
            # If relation_type column exists, include it in the description and data
            relation_info = {
                'source_symbol': source_symbol,
                'target_symbol': target_symbol
            }
            
            # Add relation type if it exists in the DataFrame
            if 'relation_type' in bmgc_relation_df.columns:
                relation_type = row['relation_type']
                relation_info['relation_type'] = relation_type
                text_descriptions[-1] = f"{source_symbol} -{relation_type}-> {target_symbol}"
                
            result_data.append(relation_info)
    
    # Create a DataFrame from the results
    result_df = pd.DataFrame(result_data)
    
    return result_df, text_descriptions

# Define the HGNC symbols
hgnc_symbols = ['SNRPD3', 'RAN', 'RPS8', 'UBL5', 'SMU1', 'RRM1', 'PSMA6', 'PSMB3', 'WEE1', 
                'PHB1', 'BANF1', 'KIF11', 'SNRPD1', 'PSMA3', 'PSMD11', 'PRPF19', 'SNRPF', 
                'RPS29', 'CDC27', 'SRSF3', 'TUBGCP2', 'ECD', 'RPS20', 'PCNA', 'PSMA7', 'CDC7', 
                'RPL17', 'GINS1', 'PHB2', 'SRSF2', 'MAD2L1', 'MED14']

# Call the function to find relationships
relationships_df, relationship_texts = find_protein_relationships(hgnc_symbols, bmgc_protein_df, bmgc_relation_df)

# You can now use both the DataFrame and text descriptions
# Example usage:
print(f"Found {len(relationship_texts)} relationships between the proteins")
print("Example relationships:")
print(relationship_texts[:5])  # Print first 5 relationships

#### 11.1.6 Formulate into json QA

In [ ]:
def qa_sample_info(sample_ach_name, selected_sample_disease_bmgc_id, k=100, top_bm=100):
    # LLM Info
    print(f"Sample ACH Name: {sample_ach_name}")
    print(f"Extracting top {k} gene information for {sample_ach_name}...")
    top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list = extract_gn_info(dti_crispr_rna_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    print(f"Extracting top {k} transcript information for {sample_ach_name}...")
    top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list = extract_ts_info(dti_crispr_rna_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    print(f"Extracting top {k} protein information for {sample_ach_name}...")
    top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list = extract_pt_info(dti_crispr_rna_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    # KG Info
    dti_edge_index_path = '../BMG/DTI_data/edge_index.npy'
    print(f"Extracting disease-related proteins index and bmgc id for {selected_sample_disease_bmgc_id} ({sample_ach_name}) ...")
    disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id = extract_kg_related_proteins(selected_sample_disease_bmgc_id, dti_edge_index_path, omics_node_index_df, nodeid_index_dict, index_nodeid_dict)
    print(f"Knowledge Graph Info: Found {len(disease_protein_index)} disease-related proteins directly connected to {selected_sample_disease_bmgc_id} and {len(ppi_nodes_index)} proteins in their PPI network")
    print(f"Mapping disease-related proteins to HGNC symbols...")
    disease_protein_hgnc_dict, disease_protein_hgnc_list = bmgc_pt_id_to_hgnc(disease_protein_bmgc_id, bmgc_protein_df)
    print(f"Mapping PPI-related proteins to HGNC symbols...")
    ppi_hgnc_dict, ppi_hgnc_list = bmgc_pt_id_to_hgnc(ppi_nodes_bmgc_id, bmgc_protein_df)
    # LLM Used KG Info
    print(f"Extracting protein relationships from BMGC...")
    # Convert the any non-existed string in top_k_gene_hgnc_name_list + top_k_transcript_hgnc_name_list + top_k_protein_hgnc_name_list + disease_protein_hgnc_list into empty list []
    if top_k_gene_hgnc_name_list == "non-existed": top_k_gene_hgnc_name_list = []
    if top_k_transcript_hgnc_name_list == "non-existed": top_k_transcript_hgnc_name_list = []
    if top_k_protein_hgnc_name_list == "non-existed": top_k_protein_hgnc_name_list = []
    if disease_protein_hgnc_list == "non-existed": disease_protein_hgnc_list = []
    # Combine all the HGNC symbols into a single list for relationship extraction
    omics_disease_protein_hgnc_list = list(set(top_k_gene_hgnc_name_list + top_k_transcript_hgnc_name_list + top_k_protein_hgnc_name_list + disease_protein_hgnc_list))
    relationships_df, relationship_texts = find_protein_relationships(omics_disease_protein_hgnc_list, bmgc_protein_df, bmgc_relation_df)
    print(f"Knowledge Graph Info: Found {len(omics_disease_protein_hgnc_list)} unique proteins and {len(relationship_texts)} relationships between them")
    # Answer Info
    print(f"Extracting top {top_bm} CRISPR gene information for {sample_ach_name}...")
    top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list = extract_answer(answer_crispr_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm)
    return top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list, \
              top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list, \
                top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list, \
                disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id, \
                    disease_protein_hgnc_dict, disease_protein_hgnc_list, ppi_hgnc_dict, ppi_hgnc_list, relationship_texts, \
                        top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
selected_sample_disease_bmgc_id = 'BMGC_DS07934'
k = 100
top_bm = 100
bmgc_protein_llmnameid_combined_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
return_tuples = qa_sample_info(sample_ach_name, selected_sample_disease_bmgc_id, k=k, top_bm=top_bm)


In [ ]:
display(dti_sample_info_index)

In [ ]:
# import json
# import os
# from tqdm import tqdm

# # Parameters
# k = 10
# top_bm = 100
# save_every_n = 10
# count = 0

# # Output folder and filename
# output_dir = "../QA_Data/"
# os.makedirs(output_dir, exist_ok=True)
# output_filename = f"multi_sample_qa_info_k{k}_bm{top_bm}.json"
# output_path = os.path.join(output_dir, output_filename)

# # Final output dictionary
# multi_sample_qa_json = {}

# # Iterate through the sample info dataframe
# for _, row in tqdm(dti_sample_info_index.iterrows(), total=len(dti_sample_info_index)):
#     count += 1

#     sample_ach_name = row["depMapID"]
#     sample_dti_index = row["Index"]
#     cell_line_name = row["Name"]
#     disease = row["BMGC_Disease_name"]
#     disease_bmgc_id = row["BMGC_Disease_ID"]

#     print(f"Processing sample: {sample_ach_name} ({cell_line_name})")
#     print(f"Sample Disease: {disease}")
#     print(f"Sample Disease BMGC ID: {disease_bmgc_id}")

#     try:
#         (top_k_gene_hgnc, top_k_gene_bmgc, top_k_gene_llm,
#         top_k_ts_hgnc, top_k_ts_bmgc, top_k_ts_llm,
#         top_k_pt_hgnc, top_k_pt_bmgc, top_k_pt_llm,
#         dis_pt_idx, dis_pt_bmgc, ppi_idx, ppi_bmgc,
#         dis_pt_hgnc_dict, dis_pt_hgnc, ppi_hgnc_dict, ppi_hgnc, relationship_texts,
#         ans_hgnc, ans_bmgc, ans_llm) = qa_sample_info(sample_ach_name, disease_bmgc_id, k=k, top_bm=top_bm)

#         multi_sample_qa_json[sample_ach_name] = {
#             "cell_line_name": cell_line_name,
#             "sample_dti_index": sample_dti_index,
#             "disease": disease,
#             "disease_bmgc_id": disease_bmgc_id,
#             "input": {
#                 "top_k_gene": {
#                     "hgnc_symbols": top_k_gene_hgnc,
#                     "protein_bmgc_ids": top_k_gene_bmgc,
#                     "protein_llmname_ids": top_k_gene_llm
#                 },
#                 "top_k_transcript": {
#                     "hgnc_symbols": top_k_ts_hgnc,
#                     "protein_bmgc_ids": top_k_ts_bmgc,
#                     "protein_llmname_ids": top_k_ts_llm
#                 },
#                 "top_k_protein": {
#                     "hgnc_symbols": top_k_pt_hgnc,
#                     "protein_bmgc_ids": top_k_pt_bmgc,
#                     "protein_llmname_ids": top_k_pt_llm
#                 },
#                 "knowledge_graph": {
#                     "disease_protein": {
#                         "bmgc_ids": dis_pt_bmgc,
#                         "hgnc_symbols": dis_pt_hgnc,
#                         "indices": dis_pt_idx
#                     },
#                     "ppi_neighbors": {
#                         "bmgc_ids": ppi_bmgc,
#                         "hgnc_symbols": ppi_hgnc,
#                         "indices": ppi_idx
#                     },
#                     "protein_relationships": relationship_texts,
#                 }
#             },
#             "ground_truth_answer": {
#                 "top_bm_gene": {
#                     "hgnc_symbols": ans_hgnc,
#                     "protein_bmgc_ids": ans_bmgc,
#                     "protein_llmname_ids": ans_llm
#                 }
#             }
#         }

#     except Exception as e:
#         print(f"⚠️ Error processing {sample_ach_name}: {e}")
#         continue

#     # Periodic save every 10 samples
#     if count % save_every_n == 0:
#         with open(output_path, "w") as f:
#             json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)
#         print(f"💾 Auto-saved JSON at {count} samples to: {output_path}")

# # Final save after loop
# with open(output_path, "w") as f:
#     json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)

# print(f"✅ Final JSON saved to: {output_path}")

In [ ]:
import json
import os
from tqdm import tqdm

# Parameters
k = 50
top_bm = 100
save_every_n = 10

# Output folder and filename
output_dir = "../QA_Data/"
os.makedirs(output_dir, exist_ok=True)
output_filename = f"multi_sample_qa_info_k{k}_bm{top_bm}.json"
output_path = os.path.join(output_dir, output_filename)

# Load existing JSON file if it exists
if os.path.exists(output_path):
    with open(output_path, "r") as f:
        multi_sample_qa_json = json.load(f)
    print(f"Loaded existing JSON with {len(multi_sample_qa_json)} processed samples")
else:
    multi_sample_qa_json = {}
    print("No existing JSON found, starting fresh")

# Load sample info data
dti_sample_info_index = pd.read_csv('../BMG/process_data/dti_combined_samples.csv')
dti_sample_index = pd.read_csv('../BMG/DTI_data/sample_index.csv')
dti_sample_info_index = pd.merge(dti_sample_info_index, dti_sample_index, 
                                how='inner', left_on='depMapID', right_on='Sample').drop(columns=['Sample'])
dti_sample_info_index = dti_sample_info_index[dti_sample_info_index['depMapID'].isin(overlapped_dti_crispr_rna_samples)].reset_index(drop=True)
dti_sample_info_index['BMGC_Disease_name'] = dti_sample_info_index['BMGC_Disease_name'].replace(r' \| ', ' or ', regex=True)

# Get the list of samples that have already been processed
processed_samples = set(multi_sample_qa_json.keys())
print(f"Found {len(processed_samples)} already processed samples")

count = len(processed_samples)
total_to_process = len(dti_sample_info_index)
remaining = total_to_process - count
print(f"Total samples to process: {total_to_process}, already processed: {count}, remaining: {remaining}")

# Iterate through the sample info dataframe, skipping already processed samples
for idx, row_tuple in tqdm(enumerate(dti_sample_info_index.iterrows()), total=len(dti_sample_info_index)):
    _, row = row_tuple  # Unpack the tuple - index and row data

    sample_ach_name = row["depMapID"]
    
    # Skip if already processed
    if sample_ach_name in processed_samples:
        continue
        
    count += 1
    sample_dti_index = row["Index"]
    cell_line_name = row["Name"]
    disease = row["BMGC_Disease_name"]
    disease_bmgc_id = row["BMGC_Disease_ID"]

    print(f"Processing sample {count}/{total_to_process}: {sample_ach_name} ({cell_line_name})")
    print(f"Sample DTI Index: {sample_dti_index}")
    print(f"Sample Disease: {disease}")
    print(f"Sample Disease BMGC ID: {disease_bmgc_id}")

    try:
        (top_k_gene_hgnc, top_k_gene_bmgc, top_k_gene_llm,
        top_k_ts_hgnc, top_k_ts_bmgc, top_k_ts_llm,
        top_k_pt_hgnc, top_k_pt_bmgc, top_k_pt_llm,
        dis_pt_idx, dis_pt_bmgc, ppi_idx, ppi_bmgc,
        dis_pt_hgnc_dict, dis_pt_hgnc, ppi_hgnc_dict, ppi_hgnc, relationship_texts,
        ans_hgnc, ans_bmgc, ans_llm) = qa_sample_info(sample_ach_name, disease_bmgc_id, k=k, top_bm=top_bm)

        multi_sample_qa_json[sample_ach_name] = {
            "cell_line_name": cell_line_name,
            "sample_dti_index": sample_dti_index,
            "disease": disease,
            "disease_bmgc_id": disease_bmgc_id,
            "input": {
                "top_k_gene": {
                    "hgnc_symbols": top_k_gene_hgnc,
                    "protein_bmgc_ids": top_k_gene_bmgc,
                    "protein_llmname_ids": top_k_gene_llm
                },
                "top_k_transcript": {
                    "hgnc_symbols": top_k_ts_hgnc,
                    "protein_bmgc_ids": top_k_ts_bmgc,
                    "protein_llmname_ids": top_k_ts_llm
                },
                "top_k_protein": {
                    "hgnc_symbols": top_k_pt_hgnc,
                    "protein_bmgc_ids": top_k_pt_bmgc,
                    "protein_llmname_ids": top_k_pt_llm
                },
                "knowledge_graph": {
                    "disease_protein": {
                        "bmgc_ids": dis_pt_bmgc,
                        "hgnc_symbols": dis_pt_hgnc,
                        "indices": dis_pt_idx
                    },
                    "ppi_neighbors": {
                        "bmgc_ids": ppi_bmgc,
                        "hgnc_symbols": ppi_hgnc,
                        "indices": ppi_idx
                    },
                    "protein_relationships": relationship_texts,
                }
            },
            "ground_truth_answer": {
                "top_bm_gene": {
                    "hgnc_symbols": ans_hgnc,
                    "protein_bmgc_ids": ans_bmgc,
                    "protein_llmname_ids": ans_llm
                }
            }
        }

    except Exception as e:
        print(f"⚠️ Error processing {sample_ach_name}: {e}")
        continue

    # Periodic save every N samples
    if count % save_every_n == 0:
        with open(output_path, "w") as f:
            json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)
        print(f"💾 Auto-saved JSON at {count}/{total_to_process} samples to: {output_path}")
        print(f"Last processed sample: {sample_ach_name}")
        processed = len(multi_sample_qa_json)
        remaining = total_to_process - processed
        print(f"Progress: {processed}/{total_to_process} ({processed/total_to_process*100:.1f}%), Remaining: {remaining}")

# Final save after loop
with open(output_path, "w") as f:
    json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)

print(f"✅ Final JSON saved to: {output_path}")
print(f"Total samples processed: {len(multi_sample_qa_json)}")

##### Make up the sample_dti_index

In [ ]:
import json
import pandas as pd

# Load the JSON file
input_path = "../QA_Data/multi_sample_qa_info_k100_bm100.json"
with open(input_path, "r") as f:
    multi_sample_qa_json = json.load(f)

print(f"Loaded JSON with {len(multi_sample_qa_json)} samples")

# Load sample info data with correct index mapping
dti_sample_info_index = pd.read_csv('../BMG/process_data/dti_combined_samples.csv')
dti_sample_index = pd.read_csv('../BMG/DTI_data/sample_index.csv')
dti_sample_info_index = pd.merge(dti_sample_info_index, dti_sample_index, 
                               how='inner', left_on='depMapID', right_on='Sample').drop(columns=['Sample'])

# Create a mapping from sample ID to index
sample_to_index_map = dict(zip(dti_sample_info_index['depMapID'], dti_sample_info_index['Index']))

# Update each sample's index in the JSON
updated_count = 0
for sample_ach_name in multi_sample_qa_json.keys():
    if sample_ach_name in sample_to_index_map:
        # Get the correct index from our mapping
        sample_dti_index = sample_to_index_map[sample_ach_name]
        
        # Update the index in the JSON
        old_index = multi_sample_qa_json[sample_ach_name]["sample_dti_index"]
        multi_sample_qa_json[sample_ach_name]["sample_dti_index"] = int(sample_dti_index)
        
        # Track changes
        if old_index != int(sample_dti_index):
            updated_count += 1
            print(f"Updated {sample_ach_name}: {old_index} -> {sample_dti_index}")
    else:
        print(f"Warning: Sample {sample_ach_name} not found in index mapping")

print(f"Updated {updated_count} sample indices")

# Save the updated JSON
output_path = "../QA_Data/multi_sample_qa_info_k100_bm100.json"
with open(output_path, "w") as f:
    json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)
print(f"Updated JSON saved to: {output_path}")

### 11.2 RNAi-QA

### 11.2.1 RNAi Answer

In [ ]:
# group by gene_name to average the values for filtered_rna_df
answer_rna_df = filtered_rna_df.groupby('gene_name').mean().reset_index()
answer_rna_df = answer_rna_df[['gene_name'] + sorted(overlapped_dti_crispr_rna_samples)].copy()
display(answer_rna_df)

In [ ]:
def extract_rna_answer(answer_rna_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm=100):
    # Extract the lowest top_bm values for the given sample name
    top_bm_genes = answer_rna_df.nsmallest(top_bm, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top bm genes by their values in ascending order
    top_bm_genes = top_bm_genes.sort_values(by=sample_ach_name, ascending=True).reset_index(drop=True)
    top_bm_gene_hgnc_name_list = top_bm_genes['gene_name'].tolist()
    # Merge with the bmgc_gene_df to get the BioMedGraphica_Conn_ID
    bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_bm_bmgc_gene_df = pd.merge(bmgc_gene_df, top_bm_genes, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_bm_bmgc_gene_protein_df = pd.merge(gene_transcript_protein_entity_df, top_bm_bmgc_gene_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_bm_bmgc_gene_protein_info_df = pd.merge(top_bm_bmgc_gene_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_bm_gene_protein_bmgc_id_list = top_bm_bmgc_gene_protein_info_df['BMGC_PT_ID'].tolist()
    top_bm_gene_protein_bmgc_llmnameid_list = top_bm_bmgc_gene_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
top_bm = 100
bmgc_protein_llmnameid_combined_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list = extract_answer(answer_rna_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm)
print(f"Top {top_bm} Gene HGNC Names:", top_bm_gene_hgnc_name_list)
print(f"Top {top_bm} Gene Protein BMGC IDs:", top_bm_gene_protein_bmgc_id_list)
print(f"Top {top_bm} Gene Protein BMGC LLM Name IDs:", top_bm_gene_protein_bmgc_llmnameid_list)

### 11.2.2 Formulate into json QA

In [ ]:
import json
# Output folder and filename
output_dir = "../QA_Data/"
os.makedirs(output_dir, exist_ok=True)
input_filename = "multi_sample_qa_info_k100_bm100.json"
input_path = os.path.join(output_dir, input_filename)
output_filename = "multi_sample_qa_info_rnai_k100_bm100.json"
output_path = os.path.join(output_dir, output_filename)

# Load existing JSON file
with open(input_path, "r") as f:
    multi_sample_qa_json = json.load(f)
print(f"Loaded JSON with {len(multi_sample_qa_json)} samples")

# Update each sample's ground truth answer with RNAi data
updated_count = 0
sample_count = len(multi_sample_qa_json)

for i, (sample_ach_name, sample_data) in enumerate(tqdm(multi_sample_qa_json.items(), total=sample_count, desc="Updating samples")):
    try:
        # Get RNAi-based answers using extract_rna_answer
        ans_hgnc, ans_bmgc, ans_llm = extract_rna_answer(
            answer_rna_df, 
            bmgc_gene_df, 
            gene_transcript_protein_entity_df, 
            bmgc_protein_llmnameid_combined_df, 
            sample_ach_name, 
            top_bm=top_bm
        )
        
        # Update the ground truth answer section
        multi_sample_qa_json[sample_ach_name]["ground_truth_answer"] = {
            "top_bm_gene": {
                "hgnc_symbols": ans_hgnc,
                "protein_bmgc_ids": ans_bmgc,
                "protein_llmname_ids": ans_llm
            }
        }
        updated_count += 1
        
        # Periodic save every 20 samples
        if (i + 1) % 20 == 0:
            with open(output_path, "w") as f:
                json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)
            print(f"Processed {i+1}/{sample_count} samples, saved intermediate results")
            
    except Exception as e:
        print(f"Error updating {sample_ach_name}: {str(e)}")

# Final save
with open(output_path, "w") as f:
    json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)

print(f"✅ Final JSON saved to: {output_path}")
print(f"Updated {updated_count} out of {sample_count} samples with RNAi data")

## 12. TargetRanker

In [ ]:
# Load training and test samples
train_dti_crispr_rna_samples_df = pd.read_csv('../QA_Data/train_dti_crispr_rna_samples.csv')
test_dti_crispr_rna_samples_df = pd.read_csv('../QA_Data/test_dti_crispr_rna_samples.csv')
# display(train_dti_crispr_rna_samples_df)
# display(test_dti_crispr_rna_samples_df)
test_dti_crispr_rna_samples = test_dti_crispr_rna_samples_df['Sample'].tolist()
print(f"Test samples: {len(test_dti_crispr_rna_samples)}")

In [ ]:
raw_gene_expression_df = pd.read_csv('./raw_data/OmicsExpressionGenesExpectedCountProfile.csv')
display(raw_gene_expression_df)

In [ ]:
# # read the map dict
# cell_line_map_dict_df = pd.read_csv('./raw_data/Model.csv')
# display(cell_line_map_dict_df.head(15))
# # create the map diict by columns ModelID and PatientID
# cell_line_map_dict = dict(zip(cell_line_map_dict_df['PatientID'], cell_line_map_dict_df['ModelID']))
# print(f"Test cell line map dict: {cell_line_map_dict['PT-gj46wT']}")

In [ ]:
# read another map dict file
profile_map_dict_df = pd.read_csv('./raw_data/OmicsProfiles.csv')
display(profile_map_dict_df.head(15))
# create the map dict by columns ProfileID and ModelID
profile_map_dict = dict(zip(profile_map_dict_df['ProfileID'], profile_map_dict_df['ModelID']))
profile_reverse_map_dict = dict(zip(profile_map_dict_df['ModelID'], profile_map_dict_df['ProfileID']))
print(f"Test profile map dict: {profile_map_dict['PR-AdBjpG']}")
print(f"Test profile reverse map dict: {profile_reverse_map_dict['ACH-000001']}")

In [ ]:
# Fill NaN values with 0
gene_expression_df = raw_gene_expression_df.fillna(0).copy()
# Renamed the column 'Unnamed: 0' to 'Gene_Symbol'
gene_expression_df.rename(columns={'Unnamed: 0': 'Samples'}, inplace=True)
# Map the conetent in column ['Samples'] to their corresponding model IDs using the profile_map_dict
gene_expression_df['Samples'] = gene_expression_df['Samples'].map(profile_map_dict)
# Remove rows with NaN values in the 'Samples' column
gene_expression_df = gene_expression_df[gene_expression_df['Samples'].notna()]

# For the column with name in format TSPAN6 (ENSG00000000003), only keep the second part ENSG00000000003 with out the brackets "()"
gene_expression_df.rename(columns=lambda x: x.split('(')[-1].split(')')[0] if '(' in x else x, inplace=True)
# Sort the DataFrame by the 'Samples' column
gene_expression_df = gene_expression_df.sort_values(by='Samples').reset_index(drop=True)

In [ ]:
# Transpose the DataFrame properly
gene_expression_t_df = gene_expression_df.set_index("Samples")
gene_expression_t_df = gene_expression_t_df.transpose()

# Reset the index and rename it to 'gene_id'
gene_expression_t_df.reset_index(inplace=True)
gene_expression_t_df.rename(columns={'index': 'gene_id'}, inplace=True)

# Fill any NaN values with 0
gene_expression_t_df = gene_expression_t_df.fillna(0.0)
display(gene_expression_t_df)

In [ ]:
# groupby the gene_name and calculate the mean
gene_expression_t_df = gene_expression_t_df.groupby('gene_id', as_index=False).mean()
# only keep the columns in the list test_dti_crispr_rna_samples
gene_expression_t_df = gene_expression_t_df[['gene_id'] + sorted(test_dti_crispr_rna_samples)].copy()
display(gene_expression_t_df)

In [ ]:
import os
# Make folder called "TargetRanker"
os.makedirs('TargetRanker', exist_ok=True)
# Save the gene_df to a csv file in the TargetRanker folder
gene_expression_t_df.to_csv('./TargetRanker/depmap_test_gene_expression.csv', index=False)

In [ ]:
os.makedirs('./TargetRanker/samples', exist_ok=True)
for sample in gene_expression_t_df.columns[1:]:
    # Create a new DataFrame for each sample
    sample_df = gene_expression_t_df[['gene_id', sample]].copy()
    # Convert integer-like floats to integers
    sample_df[sample] = sample_df[sample].apply(lambda x: int(x) if x == int(x) else x)
    # Set gene_id as index
    sample_df.set_index('gene_id', inplace=True)
    # Save with tab delimiter and correct format
    with open(f'./TargetRanker/samples/{sample}.tsv', 'w') as f:
        f.write(f'\t{sample}\n')  # Write header with tab prefix
        # Use to_csv with sep='\t' but don't include a newline at the end of each line
        sample_df.to_csv(f, sep='\t', header=False, lineterminator='\n')

##### Separate the samples by TCGA code

In [ ]:
import pandas as pd
dti_combined_samples_df = pd.read_csv('./process_data/dti_combined_samples.csv')
test_dti_crispr_rna_samples_df = pd.read_csv('./CRISPR-Graph/test_dti_crispr_rna_samples_index.csv')
test_dti_crispr_rna_samples_info_df = pd.merge(test_dti_crispr_rna_samples_df, dti_combined_samples_df, how='inner', left_on='Sample', right_on='depMapID').drop(columns=['depMapID'])
display(test_dti_crispr_rna_samples_info_df.head(3))

In [ ]:
import pandas as pd
import os

# Create output directory if it doesn't exist
os.makedirs('./TargetRanker/TCGA_Samples', exist_ok=True)

# Group by TCGA code and process each group
for tcga_code, group in test_dti_crispr_rna_samples_info_df.groupby('tcga_code'):
    if pd.isna(tcga_code):
        print(f"Skipping samples with missing TCGA code")
        continue
        
    # Get samples for this TCGA code
    tcga_samples = group['Sample'].tolist()
    print(f"Processing TCGA code {tcga_code} with {len(tcga_samples)} samples")
    
    # Filter gene expression data for these samples
    # Make sure to include the gene_id column
    sample_tcga_df = gene_expression_t_df[['gene_id'] + tcga_samples]
    
    # Create a meaningful filename
    # if tcga_code contains string "/", replace it with "_"
    tcga_code = tcga_code.replace("/", "_")
    # Save the data as TSV
    output_filename = f"./TargetRanker/TCGA_Samples/sample_{tcga_code}_{len(tcga_samples)}.tsv"
    sample_tcga_df.to_csv(output_filename, index=False, sep='\t')
    print(f"Saved {len(sample_tcga_df)} gene expressions for {len(tcga_samples)} samples to {output_filename}")

## 13. Graph CRISPR

### 13.1 CRISPR train/test

In [ ]:
train_dti_crispr_rna_samples_df = pd.read_csv('../QA_Data/train_dti_crispr_rna_samples.csv')
# display(train_dti_crispr_rna_samples_df)
test_dti_crispr_rna_samples_df = pd.read_csv('../QA_Data/test_dti_crispr_rna_samples.csv')
# display(test_dti_crispr_rna_samples_df)
dti_crispr_rna_samples_df = pd.concat([train_dti_crispr_rna_samples_df, test_dti_crispr_rna_samples_df], axis=0).reset_index(drop=True)
dti_crispr_rna_samples_df = dti_crispr_rna_samples_df.sort_values(by='Sample').reset_index(drop=True)
display(dti_crispr_rna_samples_df)
# Convert dti_crispr_rna_samples_df to a dataframe with another column called Index ranging from 0 to len(dti_crispr_rna_samples_df)-1
dti_crispr_rna_samples_df['Index'] = dti_crispr_rna_samples_df.index
# Save the dti_crispr_rna_samples_df to a csv file in the TargetRanker folder
dti_crispr_rna_samples_df.to_csv('./CRISPR-Graph/dti_crispr_rna_sample_index.csv', index=False)
display(dti_crispr_rna_samples_df)  

In [ ]:
# Select the omics data by index
dti_crispr_rna_sample_index = pd.read_csv('./CRISPR-Graph/dti_crispr_rna_sample_index.csv')
display(dti_crispr_rna_sample_index)
pretrain_sample_index = pd.read_csv('./Pretrain_data/sample_index.csv')
# rename the column Index to Pre_Index in pretrain_sample_index
pretrain_sample_index.rename(columns={'Index': 'Pre_Index'}, inplace=True)
# Merge dti_sample_index with dti_crispr_rna_sample_index on Sample column
dti_crispr_rna_sample_index = pd.merge(dti_crispr_rna_sample_index, pretrain_sample_index, how='inner', on='Sample')

# Merge pretrain_sample_index_df with train_dti_crispr_rna_samples_df and test_dti_crispr_rna_samples_df
train_dti_crispr_rna_samples_index_df = pd.merge(train_dti_crispr_rna_samples_df, dti_crispr_rna_sample_index, how='inner', on='Sample')
train_dti_crispr_rna_samples_index_df = train_dti_crispr_rna_samples_index_df.sort_values(by='Pre_Index').reset_index(drop=True)
train_dti_crispr_rna_samples_index_list = train_dti_crispr_rna_samples_index_df['Pre_Index'].tolist()
test_dti_crispr_rna_samples_index_df = pd.merge(test_dti_crispr_rna_samples_df, dti_crispr_rna_sample_index, how='inner', on='Sample')
test_dti_crispr_rna_samples_index_df = test_dti_crispr_rna_samples_index_df.sort_values(by='Index').reset_index(drop=True)
test_dti_crispr_rna_samples_index_list = test_dti_crispr_rna_samples_index_df['Pre_Index'].tolist()
dti_crispr_rna_samples_index_df = dti_crispr_rna_sample_index.copy()
dti_crispr_rna_samples_index_list = dti_crispr_rna_samples_index_df['Pre_Index'].tolist()


pretrain_bmgc_omics_array_T = np.load('./Pretrain_data/pretrain_bmgc_omics.npy')
print(f"Pretrain omics data shape: {pretrain_bmgc_omics_array_T.shape}")
# Convert train_dti_crispr_rna_samples_index_list and test_dti_crispr_rna_samples_index_list into numpy array
train_dti_crispr_rna_samples_index_array =  np.array(train_dti_crispr_rna_samples_index_list)
test_dti_crispr_rna_samples_index_array =  np.array(test_dti_crispr_rna_samples_index_list)
dti_crispr_rna_samples_index_array =  np.array(dti_crispr_rna_samples_index_list)
# Select the omics data by index
dti_train_crispr_rna_samples_omics_array = pretrain_bmgc_omics_array_T[train_dti_crispr_rna_samples_index_array, :]
print(f"Train crispr omics data shape: {dti_train_crispr_rna_samples_omics_array.shape}")
dti_test_crispr_rna_samples_omics_array = pretrain_bmgc_omics_array_T[test_dti_crispr_rna_samples_index_array, :]
print(f"Test crispr omics data shape: {dti_test_crispr_rna_samples_omics_array.shape}")
dti_crispr_rna_samples_omics_array = pretrain_bmgc_omics_array_T[dti_crispr_rna_samples_index_array, :]
print(f"All crispr omics data shape: {dti_crispr_rna_samples_omics_array.shape}")

In [ ]:
# Make folder called "CRISPR-Graph"
os.makedirs('CRISPR-Graph', exist_ok=True)
# Save train_dti_crispr_rna_samples_index_df to a csv file in the CRISPR-Graph folder
train_dti_crispr_rna_samples_index_df.to_csv('./CRISPR-Graph/train_dti_crispr_rna_samples_index.csv', index=False)
# Save test_dti_crispr_rna_samples_index_df to a csv file in the CRISPR-Graph folder
test_dti_crispr_rna_samples_index_df.to_csv('./CRISPR-Graph/test_dti_crispr_rna_samples_index.csv', index=False)
# Save dti_crispr_rna_samples_index_df to a csv file in the CRISPR-Graph folder
dti_crispr_rna_samples_index_df.to_csv('./CRISPR-Graph/dti_crispr_rna_samples_index.csv', index=False)

# Save dti_crispr_rna_samples_omics_array to a numpy file in the CRISPR-Graph folder
np.save('./CRISPR-Graph/dti_train_crispr_rna_samples_omics.npy', dti_train_crispr_rna_samples_omics_array)
np.save('./CRISPR-Graph/dti_test_crispr_rna_samples_omics.npy', dti_test_crispr_rna_samples_omics_array)
np.save('./CRISPR-Graph/dti_crispr_rna_samples_omics.npy', dti_crispr_rna_samples_omics_array)

### 13.2 CRISPR node and edge information

In [ ]:
omics_nodes_index_data_df = pd.read_csv('./Pretrain_data/omics_nodes_index.csv')
display(omics_nodes_index_data_df.head(5))
# Save the omics_nodes_index_data_df to a csv file in the CRISPR-Graph folder
omics_nodes_index_data_df.to_csv('./CRISPR-Graph/omics_nodes_index.csv', index=False)

In [ ]:
# Create omics_node_index_dict
omics_node_index_dict = dict(zip(omics_nodes_index_data_df['Node'], omics_nodes_index_data_df['Index']))
print(f"Omics node index dict: {omics_node_index_dict['BMGC_PM00001']}")

In [ ]:
bmgc_relation_df = pd.read_csv('./BioMedGraphica-Conn/Relation/BioMedGraphica_Conn_Relation.csv')
# filter our the rows with Relation type in the list ['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein', 'Protein-Protein']
bmgc_omics_relation_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein', 'Protein-Protein'])].reset_index(drop=True)
# check if there is any null values in the bmgc_omics_relation_df
print(bmgc_omics_relation_df.isnull().sum())
display(bmgc_omics_relation_df)

In [ ]:
# Convert the bmgc_omics_relation_df to numpy array and save it to the Pretrain_data folder as the edge_index.npy
# Keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
edge_index_df = bmgc_omics_relation_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()

# Map the BMGC_From_ID and BMGC_To_ID by the omics_node_index_dict
edge_index_df['BMGC_From_ID'] = edge_index_df['BMGC_From_ID'].map(omics_node_index_dict)
edge_index_df['BMGC_To_ID'] = edge_index_df['BMGC_To_ID'].map(omics_node_index_dict)

# check the null values in the edge_index_df
print("Null values in edge_index_df:")
print(edge_index_df.isnull().sum())
# print the rows with null values in column BMGC_From_ID the edge_index_df
print(edge_index_df[edge_index_df['BMGC_From_ID'].isnull()])
display(edge_index_df)
# convert the edge_index_df to numpy array and save it to the Pretrain_data folder
edge_index_array = edge_index_df.to_numpy().T
print('The shape of edge_index_array is:', edge_index_array.shape)
print(edge_index_array[:10])  # Print first 10 rows as a sample
# save the numpy array to the Pretrain_data folder
np.save('./CRISPR-Graph/edge_index.npy', edge_index_array)

In [ ]:
# filter out the internal_edge_index by Type belongs to ['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein']
internal_edge_index_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'].isin(['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein'])].reset_index(drop=True)
internal_edge_index_df = internal_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
internal_edge_index_df['BMGC_From_ID'] = internal_edge_index_df['BMGC_From_ID'].map(node_index_dict)
internal_edge_index_df['BMGC_To_ID'] = internal_edge_index_df['BMGC_To_ID'].map(node_index_dict)
display(internal_edge_index_df)
# convert the internal_edge_index_df to numpy array and save it to the Pretrain_data folder
internal_edge_index_array = internal_edge_index_df.to_numpy().T
print('The shape of internal_edge_index_array is:', internal_edge_index_array.shape)
print(internal_edge_index_array[:10])  # Print first 10 rows as a sample
# save the numpy array to the Pretrain_data folder
np.save('./CRISPR-Graph/internal_edge_index.npy', internal_edge_index_array)

In [ ]:
# generate the ppi_edge_index by selecting bmgc_relation_df in Type ['Protein-Protein']
ppi_edge_index_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Protein-Protein'])].copy()
# keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
ppi_edge_index_df = ppi_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
ppi_edge_index_df['BMGC_From_ID'] = ppi_edge_index_df['BMGC_From_ID'].map(node_index_dict)
ppi_edge_index_df['BMGC_To_ID'] = ppi_edge_index_df['BMGC_To_ID'].map(node_index_dict)
# check the null values in the ppi_edge_index_df
print("Null values in ppi_edge_index_df:")
print(ppi_edge_index_df.isnull().sum())
# convert the ppi_edge_index_df to numpy array and save it to the DTI_data folder
ppi_edge_index_array = ppi_edge_index_df.to_numpy().T
print('The shape of ppi_edge_index_array is:', ppi_edge_index_array.shape)
print(ppi_edge_index_array)
# save the numpy array to the DTI_data folder
np.save('./CRISPR-Graph/ppi_edge_index.npy', ppi_edge_index_array)

### 13.3 CRISPR label

#### 13.3.1 Build up internal relation for KO drop

In [ ]:
bmgc_promoter_gene_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Promoter-Gene'].drop(columns=['Type'])
bmgc_gene_transcript_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Gene-Transcript'].drop(columns=['Type'])
bmgc_transcript_protein_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Transcript-Protein'].drop(columns=['Type'])

In [ ]:
# Rename the columns
bmgc_promoter_gene_df.rename(columns={'BMGC_From_ID':'promoterID','BMGC_To_ID':'geneID'}, inplace=True)
bmgc_gene_transcript_df.rename(columns={'BMGC_From_ID':'geneID','BMGC_To_ID':'transcriptID'}, inplace=True)
bmgc_transcript_protein_df.rename(columns={'BMGC_From_ID':'transcriptID','BMGC_To_ID':'proteinID'}, inplace=True)

In [ ]:
# Merge gene_transcript and transcript_protein dataframes
bmgc_gene_transcript_protein_df = bmgc_transcript_protein_df.merge(bmgc_gene_transcript_df, on='transcriptID',how='outer')
display(bmgc_gene_transcript_protein_df)
# Merge promoter
bmgc_promoter_gene_transcript_protein_df = bmgc_gene_transcript_protein_df.merge(bmgc_promoter_gene_df, on='geneID',how='outer')
display(bmgc_promoter_gene_transcript_protein_df)

In [ ]:
internal_relation_df = bmgc_promoter_gene_transcript_protein_df.dropna(subset=['geneID'])
internal_relation_df = internal_relation_df[['promoterID', 'geneID', 'transcriptID', 'proteinID']].copy()
display(internal_relation_df)

#### 13.3.2 Build up label

In [ ]:
import pandas as pd
raw_crispr_df = pd.read_csv('./raw_data/CRISPRGeneEffect.csv')
# get the column names of the raw_crispr_df aside from the first column and convert this to a list
first_column_name = raw_crispr_df.columns[0]
raw_crispr_df.rename(columns={first_column_name: 'Sample'}, inplace=True)
display(raw_crispr_df)

In [ ]:
train_dti_crispr_rna_samples_index_df = pd.read_csv('./CRISPR-Graph/train_dti_crispr_rna_samples_index.csv')
#display(train_dti_crispr_rna_samples_df)
test_dti_crispr_rna_samples_index_df = pd.read_csv('./CRISPR-Graph/test_dti_crispr_rna_samples_index.csv')
#display(test_dti_crispr_rna_samples_df)
train_crispr_score_df = raw_crispr_df.merge(train_dti_crispr_rna_samples_index_df, on="Sample", how="inner").drop(columns=['Index', 'Pre_Index'])
# display(train_crispr_score_df)
test_crispr_score_df = raw_crispr_df.merge(test_dti_crispr_rna_samples_index_df, on="Sample", how="inner").drop(columns=['Index', 'Pre_Index'])
# display(test_crispr_score_df)

# Set Index column to str to keep digit without .00
train_crispr_score_t_df = train_crispr_score_df.set_index("Sample").T.reset_index()
train_crispr_score_t_df.rename(columns={"index": "HGNC_Symbol"}, inplace=True)
display(train_crispr_score_t_df)
test_crispr_score_t_df = test_crispr_score_df.set_index("Sample").T.reset_index()
test_crispr_score_t_df.rename(columns={"index": "HGNC_Symbol"}, inplace=True)
display(test_crispr_score_t_df)

In [ ]:
# Check the statistics of the train_crispr_score_t_df and test_crispr_score_t_df
print("Train CRISPR score statistics:")
print(train_crispr_score_t_df.describe())
print("Test CRISPR score statistics:")
print(test_crispr_score_t_df.describe())

# Normalize the train_crispr_score_t_df and test_crispr_score_t_df by Z-score over each column
train_crispr_score_t_df.iloc[:, 1:] = (train_crispr_score_t_df.iloc[:, 1:] - train_crispr_score_t_df.iloc[:, 1:].mean()) / train_crispr_score_t_df.iloc[:, 1:].std()
test_crispr_score_t_df.iloc[:, 1:] = (test_crispr_score_t_df.iloc[:, 1:] - test_crispr_score_t_df.iloc[:, 1:].mean()) / test_crispr_score_t_df.iloc[:, 1:].std()
# Recheck the statistics of the train_crispr_score_t_df and test_crispr_score_t_df
print("Train CRISPR score statistics after normalization:")
print(train_crispr_score_t_df.describe())
print("Test CRISPR score statistics after normalization:")
print(test_crispr_score_t_df.describe())

In [ ]:
bmgc_gene_df = pd.read_csv('./BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')

import re
def extract_gene_name(gene):
    return re.sub(r"\s*\(.*?\)", "", str(gene))

train_crispr_score_t_df["HGNC_Symbol"] = train_crispr_score_t_df["HGNC_Symbol"].apply(extract_gene_name)

unique_Gene_map = bmgc_gene_df[['HGNC_Symbol']].drop_duplicates()
unique_omics_Gene = train_crispr_score_t_df[['HGNC_Symbol']].drop_duplicates()
# match
intersection = set(unique_Gene_map['HGNC_Symbol'])&set(unique_omics_Gene['HGNC_Symbol'])
total_Gene = len(unique_Gene_map)

match_rate = len(intersection) / total_Gene if total_Gene > 0 else 0
#####

print(f"match rate:{match_rate:.2%}")

train_crispr_score_bmgc_df = bmgc_gene_df.merge(train_crispr_score_t_df, on="HGNC_Symbol", how="inner")

train_crispr_score_bmgc_df = train_crispr_score_bmgc_df.drop(columns=['BioMedGraphica_ID','NCBI_Gene_ID','Ensembl_Gene_ID_Version','Gene_Start', 'Gene_End','Chromosome','Gene_Type','Ensembl_Gene_ID','HGNC_ID','Gene_Name','RefSeq_ID','OMIM_ID','HGNC_Symbol'])

display(train_crispr_score_bmgc_df)

In [ ]:
test_crispr_score_t_df["HGNC_Symbol"] = test_crispr_score_t_df["HGNC_Symbol"].apply(extract_gene_name)

unique_Gene_map = bmgc_gene_df[['HGNC_Symbol']].drop_duplicates()
unique_omics_Gene = test_crispr_score_t_df[['HGNC_Symbol']].drop_duplicates()
# match
intersection = set(unique_Gene_map['HGNC_Symbol'])&set(unique_omics_Gene['HGNC_Symbol'])
total_Gene = len(unique_Gene_map)

match_rate = len(intersection) / total_Gene if total_Gene > 0 else 0
#####

print(f"match rate:{match_rate:.2%}")

test_crispr_score_bmgc_df = bmgc_gene_df.merge(test_crispr_score_t_df, on="HGNC_Symbol", how="inner")

test_crispr_score_bmgc_df = test_crispr_score_bmgc_df.drop(columns=['BioMedGraphica_ID','NCBI_Gene_ID','Ensembl_Gene_ID_Version','Gene_Start', 'Gene_End','Chromosome','Gene_Type','Ensembl_Gene_ID','HGNC_ID','Gene_Name','RefSeq_ID','OMIM_ID','HGNC_Symbol'])

display(test_crispr_score_bmgc_df)

In [ ]:
train_crispr_score_label = train_crispr_score_bmgc_df.copy()
train_crispr_score_label_melted = train_crispr_score_label.melt(id_vars=["BioMedGraphica_Conn_ID"], var_name="ACH_ID", value_name="Value")
train_crispr_score_label_melted = train_crispr_score_label_melted.dropna(subset=['Value'])
display(train_crispr_score_label_melted)

In [ ]:
test_crispr_score_label = test_crispr_score_bmgc_df.copy()
test_crispr_score_label_melted = test_crispr_score_label.melt(id_vars=["BioMedGraphica_Conn_ID"], var_name="ACH_ID", value_name="Value")
test_crispr_score_label_melted = test_crispr_score_label_melted.dropna(subset=['Value'])
display(test_crispr_score_label_melted)

##### Merge into internal relation

In [ ]:
# Prepare the data into the index to fetch
# Based on dti_sample_index, convert this to a dict
crispr_sample_index_dict = dict(zip(dti_crispr_rna_sample_index['Sample'], dti_crispr_rna_sample_index['Index']))
# Based on omics_nodes_index_data_df, convert this to a dict
omics_nodes_index_dict = dict(zip(omics_nodes_index_data_df['Node'], omics_nodes_index_data_df['Index']))

# Map columns ['ACH_ID', 'BioMedGraphica_Conn_ID'] by the crispr_sample_index_dict and omics_nodes_index_dict
train_crispr_score_label_melted['ACH_ID'] = train_crispr_score_label_melted['ACH_ID'].map(crispr_sample_index_dict)
train_crispr_score_label_melted['BioMedGraphica_Conn_ID'] = train_crispr_score_label_melted['BioMedGraphica_Conn_ID'].map(omics_nodes_index_dict)
test_crispr_score_label_melted['ACH_ID'] = test_crispr_score_label_melted['ACH_ID'].map(crispr_sample_index_dict)
test_crispr_score_label_melted['BioMedGraphica_Conn_ID'] = test_crispr_score_label_melted['BioMedGraphica_Conn_ID'].map(omics_nodes_index_dict)

In [ ]:
# Map internal_relation_df columns ['promoterID', 'geneID', 'transcriptID', 'proteinID'] by the omics_nodes_index_dict
internal_relation_map_df = internal_relation_df.copy()
internal_relation_map_df['promoterID'] = internal_relation_map_df['promoterID'].map(omics_nodes_index_dict)
internal_relation_map_df['geneID'] = internal_relation_map_df['geneID'].map(omics_nodes_index_dict)
internal_relation_map_df['transcriptID'] = internal_relation_map_df['transcriptID'].map(omics_nodes_index_dict)
internal_relation_map_df['proteinID'] = internal_relation_map_df['proteinID'].map(omics_nodes_index_dict)

In [ ]:
ko_internal_relation_map_df = internal_relation_map_df.copy()
display(ko_internal_relation_map_df)
ko_internal_relation_index_df = ko_internal_relation_map_df.groupby(["promoterID", "geneID"], as_index=False).agg({
    "transcriptID": lambda x: list(set(x.dropna())), 
    "proteinID": lambda x: list(set(x.dropna()))  
})

In [ ]:
# Define a function to remove .0 suffix from numbers in lists
def remove_decimal_suffix(list_with_decimals):
    if isinstance(list_with_decimals, list):
        return [int(x) if not pd.isna(x) else x for x in list_with_decimals]
    return list_with_decimals

# Apply the function to both columns
ko_internal_relation_index_df['transcriptID'] = ko_internal_relation_index_df['transcriptID'].apply(remove_decimal_suffix)
ko_internal_relation_index_df['proteinID'] = ko_internal_relation_index_df['proteinID'].apply(remove_decimal_suffix)

# Display the result to verify
display(ko_internal_relation_index_df.head())


In [ ]:
train_crispr_score_label_melted_ko = train_crispr_score_label_melted.merge(ko_internal_relation_index_df, left_on='BioMedGraphica_Conn_ID',right_on='geneID' ,how='left')
train_crispr_score_label_melted_ko = train_crispr_score_label_melted_ko.drop(columns=['BioMedGraphica_Conn_ID'])
display(train_crispr_score_label_melted_ko)

# Create a new column 'merged_ids' by combining the columns into lists
train_crispr_score_label_melted_ko['merged_ids'] = train_crispr_score_label_melted_ko.apply(
    lambda row: [row['promoterID'], row['geneID']] + 
                (row['transcriptID'] if isinstance(row['transcriptID'], list) else []) +
                (row['proteinID'] if isinstance(row['proteinID'], list) else []),
    axis=1
)

# Display the result
train_crispr_score_label_melted_ko = train_crispr_score_label_melted_ko.drop(columns=['promoterID', 'geneID', 'transcriptID', 'proteinID'])
display(train_crispr_score_label_melted_ko)

In [ ]:
# reorder the columns to ['ACH_ID', 'merged_ids', 'Value']
train_crispr_score_label_melted_ko = train_crispr_score_label_melted_ko[['ACH_ID', 'merged_ids', 'Value']].copy()
display(train_crispr_score_label_melted_ko)
# Convert train_crispr_score_label_melted_ko to numpy array and save it to the CRISPR-Graph folder
train_crispr_score_label_melted_ko_array = train_crispr_score_label_melted_ko.to_numpy()
print('The shape of train_crispr_score_label_melted_ko_array is:', train_crispr_score_label_melted_ko_array.shape)
print(train_crispr_score_label_melted_ko_array[:10])  # Print first 10 rows as a sample
# Save the numpy array to the CRISPR-Graph folder
np.save('./CRISPR-Graph/train_crispr_score.npy', train_crispr_score_label_melted_ko_array)


In [ ]:
test_crispr_score_label_melted_ko = test_crispr_score_label_melted.merge(ko_internal_relation_index_df, left_on='BioMedGraphica_Conn_ID',right_on='geneID' ,how='left')
test_crispr_score_label_melted_ko = test_crispr_score_label_melted_ko.drop(columns=['BioMedGraphica_Conn_ID'])
display(test_crispr_score_label_melted_ko)

# Create a new column 'merged_ids' by combining the columns into lists
test_crispr_score_label_melted_ko['merged_ids'] = test_crispr_score_label_melted_ko.apply(
    lambda row: [row['promoterID'], row['geneID']] + 
                (row['transcriptID'] if isinstance(row['transcriptID'], list) else []) +
                (row['proteinID'] if isinstance(row['proteinID'], list) else []),
    axis=1
)

# Display the result
test_crispr_score_label_melted_ko = test_crispr_score_label_melted_ko.drop(columns=['promoterID', 'geneID', 'transcriptID', 'proteinID'])
display(test_crispr_score_label_melted_ko)

In [ ]:
# reorder the columns to ['ACH_ID', 'merged_ids', 'Value']
test_crispr_score_label_melted_ko = test_crispr_score_label_melted_ko[['ACH_ID', 'merged_ids', 'Value']].copy()
display(test_crispr_score_label_melted_ko)
# Convert test_crispr_score_label_melted_ko to numpy array and save it to the CRISPR-Graph folder
test_crispr_score_label_melted_ko_array = test_crispr_score_label_melted_ko.to_numpy()
print('The shape of test_crispr_score_label_melted_ko_array is:', test_crispr_score_label_melted_ko_array.shape)
print(test_crispr_score_label_melted_ko_array[:10])  # Print first 10 rows as a sample
# Save the numpy array to the CRISPR-Graph folder
np.save('./CRISPR-Graph/test_crispr_score.npy', test_crispr_score_label_melted_ko_array)